In [1]:
import pyodbc
import pandas as pd
from tqdm import tqdm as tqdm
from configparser import ConfigParser


In [2]:
try:
    cd /notebooks/notebooks
except:
    %cd /notebooks/notebooks
    

[WinError 3] The system cannot find the path specified: '/notebooks/notebooks'
C:\Users\FIU\Desktop\DEB_RISK_SUMMARY


In [3]:
config = ConfigParser()
config.read(r'config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [4]:
connection_string

'DRIVER=SQL SERVER;SERVER=172.16.20.43;PORT=1433;DATABASE=FINCORE_DB;UID=Fincore_Analytics_USR;PWD=F!NCore@123;TDS_Version=7.2;'

In [5]:

# def rules_triggered(a,b):
#     result = [i for i in a if i in b]
#     return result


# def intersection(case_entities,flagged_person):
#     intersection = [value for value in case_entities if value in flagged_person]
#     return intersection

In [6]:
pd.set_option("display.max_column",200)
pd.set_option("display.float_format",lambda x:'%.5f' %x)

In [7]:

# config = ConfigParser()
config.read(r'config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [22]:
# Changed Code for audit integration

conn.execute("""insert into FINCORE_DB.Fincore_Analytics.AUDIT_INTGN (JOB_RUN_ID,JOB_NAME,JOB_START_DATE,JOB_STATUS,TARGET_TBL,SOURCE_TBL,SOURCE_MAX_DATE)values
(COALESCE((select (max(JOB_RUN_ID)+1) from FINCORE_DB.Fincore_Analytics.AUDIT_INTGN),1), 'RISK_SUMMARY',GETDATE(),'Error','RISK_SUMMARY','CASE_MASTER',(SELECT MAX(T.MAX_DATE) FROM (SELECT MAX(UPDATE_DATE) AS MAX_DATE FROM FINCORE_DB.FINCORE.CASE_MASTER UNION SELECT MAX(CREATE_DATE) AS MAX_DATE FROM FINCORE_DB.FINCORE.CASE_MASTER)T))""")

job_id = pd.read_sql_query("select max(JOB_RUN_ID) as job_run_id from FINCORE_DB.Fincore_Analytics.AUDIT_INTGN",conn)
job_id = job_id.iloc[0,0]
conn.commit()



In [23]:
cursor.execute("delete  from FINCORE_DB.Fincore_Analytics.risk_summary")
conn.commit()

In [24]:
# rules_df = pd.read_sql_query("select * from FINCORE_DB.Fincore_Analytics.RISK_LKP_RULES where  is_active_flag = 1 ",conn)


In [25]:


rules_df = pd.read_sql_query("select * from FINCORE_DB.Fincore_Analytics.RISK_LKP_RULES with (nolock) where  is_active_flag = 1 ",conn)



# cmids = pd.read_sql_query("Select Case_Master_Id from FINCORE_DB.Fincore.Case_master where is_active_flag = 1 ",conn)


cmids = pd.read_sql_query('''SELECT Case_Master_Id FROM FINCORE_DB.Fincore.Case_master with (nolock) WHERE Is_Active_Flag=1 
AND coalesce(Update_Date,create_date) > COALESCE( (select MAX(SOURCE_MAX_DATE) from FINCORE_DB.Fincore_Analytics.AUDIT_INTGN with (nolock)
where JOB_STATUS='Success' and JOB_NAME='Risk_Summary' )
,cast('1900-01-01 00:00:00.0000000' as datetime2))''',conn)


cmids = cmids['Case_Master_Id'].to_list()

In [26]:
no_of_records = len(cmids)

In [27]:
no_of_records

772

In [28]:
%store job_id

Stored 'job_id' (int64)


In [ ]:
cmids =  pd.read_sql_query('''Select ''',conn)

In [ ]:
for cmid in cmids:

    s = ''
    case_entities = pd.read_sql_query("select CASE_MASTER_ID,ENTITY_MASTER_ID,Is_Main_Entity_Flag,ENTITY_TYPE from FINCORE_DB.Fincore.Case_Entity where Is_Active_Flag= 1 and Case_Master_Id = '{}'".format(cmid),conn)
    risk_entity = pd.read_sql_query("WITH T AS (SELECT B.CASE_MASTER_ID,B.ENTITY_MASTER_ID,B.ENTITY_TYPE,A.RISK_SCORE FROM FINCORE_DB.FINCORE.CASE_ENTITY B LEFT JOIN FINCORE_DB.FINCORE_ANALYTICS.RISK_PERSON A ON A.PERSON_MASTER_ID = B.ENTITY_MASTER_ID WHERE B.ENTITY_TYPE = 'I' AND  CASE_MASTER_ID = '{}' and Is_Main_Entity_Flag = 1 UNION  SELECT B.CASE_MASTER_ID,B.ENTITY_MASTER_ID,B.ENTITY_TYPE,A.RISK_SCORE FROM FINCORE_DB.FINCORE.CASE_ENTITY B LEFT JOIN FINCORE_DB.FINCORE_ANALYTICS.RISK_ORGANIZATION A  ON A.ORGANIZATION_MASTER_ID = B.ENTITY_MASTER_ID WHERE B.ENTITY_TYPE = 'O' AND CASE_MASTER_ID = '{}' and Is_Main_Entity_Flag = 1) SELECT  T.ENTITY_MASTER_ID,T.ENTITY_TYPE,coalesce(T.RISK_SCORE,0) as RISK_SCORE FROM T ".format(cmid,cmid),conn)
    main_entity = case_entities[case_entities['Is_Main_Entity_Flag'] == 1][['ENTITY_MASTER_ID','ENTITY_TYPE']].values.tolist()
#     other_entity_df = case_entities[case_entities['Is_Main_Entity_Flag'] == 0][['ENTITY_MASTER_ID','ENTITY_TYPE']].values.tolist()
    other_entity = case_entities[case_entities['Is_Main_Entity_Flag'] == 0][['ENTITY_MASTER_ID','ENTITY_TYPE']].values.tolist()

    try:
        RISK_SCORE = risk_entity['RISK_SCORE'].to_list()
        RISK_SCORE  = max(RISK_SCORE)
    except IndexError:
        RISK_SCORE = 0
    except ValueError:
        RISK_SCORE = 0

    if RISK_SCORE == 0:
        s = s+'<html> There is no main entity. </html>'

    else:

        auto_high_df = pd.read_sql_query('''with cte as(select PERSON_MASTER_ID,RULE_NO,RISK_SCORE_STATIC from FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG_RULES union  select PERSON_MASTER_ID,RULE_NO,THRESHOLD_FLG as RISK_SCORE_STATIC  from FINCORE_DB.Fincore_Analytics.RISK_PERSON_analytical_RULES   ), cte1 as(select ORGANIZATION_MASTER_ID,RULE_NO,RISK_SCORE_STATIC from FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_SUMMARY_STG_RULES union  select ORGANIZATION_MASTER_ID,RULE_NO,THRESHOLD_FLG as RISK_SCORE_STATIC from FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_analytical_RULES ) select DISTINCT Case_Master_Id,Entity_Master_Id,Entity_Type,RULE_DESC from FINCORE_DB.Fincore.Case_Entity CE  left join CTE ON CE.Entity_Master_Id = CTE.PERSON_MASTER_ID INNER JOIN FINCORE_DB.Fincore_Analytics.RISK_LKP_RULES RLR ON RLR.RULE_NO = cte.RULE_NO WHERE case_master_id = '{}' AND CE.Entity_Type = 'I' AND  risk_score_static is not null and RISK_SCORE_STATIC <> 0 and RISK_TYPE = 'Auto High Rules ' UNION ALL  select DISTINCT Case_Master_Id,Entity_Master_Id,Entity_Type,RULE_DESC from FINCORE_DB.Fincore.Case_Entity CE left join CTE1 ON CE.Entity_Master_Id = CTE1.ORGANIZATION_MASTER_ID INNER JOIN FINCORE_DB.Fincore_Analytics.RISK_LKP_RULES RLR ON RLR.RULE_NO = cte1.RULE_NO WHERE CE.Entity_Type =  'O' AND  case_master_id = '{}' AND risk_score_static is not null and RISK_SCORE_STATIC <> 0   and RISK_TYPE = 'Auto High Rules ' '''.format(cmid,cmid),conn)
        auto_df_main = pd.read_sql_query('''with cte as(select PERSON_MASTER_ID,RULE_NO,RISK_SCORE_STATIC from FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG_RULES union select PERSON_MASTER_ID,RULE_NO,THRESHOLD_FLG as RISK_SCORE_STATIC from FINCORE_DB.Fincore_Analytics.RISK_PERSON_analytical_RULES   ),cte1 as(select ORGANIZATION_MASTER_ID,RULE_NO,RISK_SCORE_STATIC from FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_SUMMARY_STG_RULES union  select ORGANIZATION_MASTER_ID,RULE_NO,THRESHOLD_FLG as RISK_SCORE_STATIC from FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_analytical_RULES )select DISTINCT Case_Master_Id,Entity_Master_Id,Entity_Type,RULE_DESC from FINCORE_DB.Fincore.Case_Entity CE  left join CTE ON CE.Entity_Master_Id = CTE.PERSON_MASTER_ID INNER JOIN FINCORE_DB.Fincore_Analytics.RISK_LKP_RULES RLR ON RLR.RULE_NO = cte.RULE_NO WHERE case_master_id = '{}' AND CE.Entity_Type = 'I' AND  risk_score_static is not null and RISK_SCORE_STATIC <> 0  and RISK_TYPE = 'Auto High Rules ' and CE.Is_Main_Entity_Flag = 1 UNION ALL  select DISTINCT Case_Master_Id,Entity_Master_Id,Entity_Type,RULE_DESC  from FINCORE_DB.Fincore.Case_Entity CE left join CTE1 ON CE.Entity_Master_Id = CTE1.ORGANIZATION_MASTER_ID INNER JOIN FINCORE_DB.Fincore_Analytics.RISK_LKP_RULES RLR ON RLR.RULE_NO = cte1.RULE_NO WHERE CE.Entity_Type =  'O' AND  case_master_id = '{}' AND risk_score_static is not null and RISK_SCORE_STATIC <> 0   and RISK_TYPE = 'Auto High Rules ' and CE.Is_Main_Entity_Flag = 1'''.format(cmid,cmid),conn)
        auto_df_main = auto_df_main['RULE_DESC'].values.tolist()
        auto_df_other = pd.read_sql_query('''with cte as(select PERSON_MASTER_ID,RULE_NO,RISK_SCORE_STATIC from FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG_RULES union select PERSON_MASTER_ID,RULE_NO,THRESHOLD_FLG as RISK_SCORE_STATIC from FINCORE_DB.Fincore_Analytics.RISK_PERSON_analytical_RULES   ),cte1 as(select ORGANIZATION_MASTER_ID,RULE_NO,RISK_SCORE_STATIC from FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_SUMMARY_STG_RULES union  select ORGANIZATION_MASTER_ID,RULE_NO,THRESHOLD_FLG as RISK_SCORE_STATIC from FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_analytical_RULES )select DISTINCT Case_Master_Id,Entity_Master_Id,Entity_Type,RULE_DESC from FINCORE_DB.Fincore.Case_Entity CE  left join CTE ON CE.Entity_Master_Id = CTE.PERSON_MASTER_ID INNER JOIN FINCORE_DB.Fincore_Analytics.RISK_LKP_RULES RLR ON RLR.RULE_NO = cte.RULE_NO WHERE case_master_id = '{}' AND CE.Entity_Type = 'I' AND  risk_score_static is not null and RISK_SCORE_STATIC <> 0  and RISK_TYPE = 'Auto High Rules ' and CE.Is_Main_Entity_Flag = 0 UNION ALL  select DISTINCT Case_Master_Id,Entity_Master_Id,Entity_Type,RULE_DESC  from FINCORE_DB.Fincore.Case_Entity CE left join CTE1 ON CE.Entity_Master_Id = CTE1.ORGANIZATION_MASTER_ID INNER JOIN FINCORE_DB.Fincore_Analytics.RISK_LKP_RULES RLR ON RLR.RULE_NO = cte1.RULE_NO WHERE CE.Entity_Type =  'O' AND  case_master_id = '{}' AND risk_score_static is not null and RISK_SCORE_STATIC <> 0   and RISK_TYPE = 'Auto High Rules ' and CE.Is_Main_Entity_Flag = 0 '''.format(cmid,cmid),conn)
        auto_df_other = auto_df_other['RULE_DESC'].values.tolist()



        watchlist = pd.read_sql_query('''WITH CTE AS(SELECT PERSON_MASTER_ID AS ENTITY_MASTER_ID,B.ENTITY_TYPE,STRING_AGG(RULE_VALUE,',') WITHIN GROUP (ORDER BY RULE_NO) AS RULE_VALUE FROM FINCORE_DB.FINCORE_ANALYTICS.RISK_PERSON_SUMMARY_STG_RULES  A INNER JOIN FINCORE_DB.FINCORE.CASE_ENTITY B ON A.PERSON_MASTER_ID = B.ENTITY_MASTER_ID WHERE RULE_NO IN ('P001001000', 'P001003000') AND ENTITY_TYPE = 'I' AND CASE_MASTER_ID = '{}' GROUP BY PERSON_MASTER_ID,RULE_NO,ENTITY_TYPE  UNION  SELECT ORGANIZATION_MASTER_ID AS ENTITY_MASTER_ID,B.ENTITY_TYPE,STRING_AGG(RULE_VALUE,',') WITHIN GROUP (ORDER BY RULE_NO)   AS RULE_VALUE FROM FINCORE_DB.FINCORE_ANALYTICS.RISK_ORGANIZATION_SUMMARY_STG_RULES A INNER JOIN FINCORE_DB.FINCORE.CASE_ENTITY B ON A.ORGANIZATION_MASTER_ID = B.ENTITY_MASTER_ID WHERE RULE_NO IN ('O001001000', 'O001003000') AND ENTITY_TYPE = 'O' AND CASE_MASTER_ID = '{}' GROUP BY ORGANIZATION_MASTER_ID,RULE_NO,ENTITY_TYPE ) SELECT ENTITY_MASTER_ID,Entity_Type,RULE_VALUE FROM CTE'''.format(cmid,cmid),conn)
        main_entity_watchlist = pd.read_sql_query('''WITH CTE AS(SELECT PERSON_MASTER_ID AS ENTITY_MASTER_ID,B.ENTITY_TYPE,STRING_AGG(RULE_VALUE,',') WITHIN GROUP (ORDER BY RULE_NO) AS RULE_VALUE FROM FINCORE_DB.FINCORE_ANALYTICS.RISK_PERSON_SUMMARY_STG_RULES  A INNER JOIN FINCORE_DB.FINCORE.CASE_ENTITY B ON A.PERSON_MASTER_ID = B.ENTITY_MASTER_ID WHERE RULE_NO IN ('P001001000', 'P001003000') AND ENTITY_TYPE = 'I' AND CASE_MASTER_ID = '{}' AND IS_MAIN_ENTITY_FLAG = 1 GROUP BY PERSON_MASTER_ID,RULE_NO,ENTITY_TYPE  UNION  SELECT ORGANIZATION_MASTER_ID AS ENTITY_MASTER_ID,B.ENTITY_TYPE,STRING_AGG(RULE_VALUE,',') WITHIN GROUP (ORDER BY RULE_NO)   AS RULE_VALUE FROM FINCORE_DB.FINCORE_ANALYTICS.RISK_ORGANIZATION_SUMMARY_STG_RULES A INNER JOIN FINCORE_DB.FINCORE.CASE_ENTITY B ON A.ORGANIZATION_MASTER_ID = B.ENTITY_MASTER_ID WHERE RULE_NO IN ('O001001000', 'O001003000') AND ENTITY_TYPE = 'O' AND CASE_MASTER_ID = '{}' AND IS_MAIN_ENTITY_FLAG = 1  GROUP BY ORGANIZATION_MASTER_ID,RULE_NO,ENTITY_TYPE ) SELECT ENTITY_MASTER_ID,ENTITY_TYPE,RULE_VALUE FROM CTE'''.format(cmid,cmid),conn)
        main_entity_watchlist = main_entity_watchlist['RULE_VALUE'].to_list()
        other_entity_watchlist = pd.read_sql_query('''WITH CTE AS(SELECT PERSON_MASTER_ID AS ENTITY_MASTER_ID,B.ENTITY_TYPE,STRING_AGG(RULE_VALUE,',') WITHIN GROUP (ORDER BY RULE_NO) AS RULE_VALUE FROM FINCORE_DB.FINCORE_ANALYTICS.RISK_PERSON_SUMMARY_STG_RULES  A INNER JOIN FINCORE_DB.FINCORE.CASE_ENTITY B ON A.PERSON_MASTER_ID = B.ENTITY_MASTER_ID WHERE RULE_NO IN ('P001001000', 'P001003000') AND ENTITY_TYPE = 'I' AND CASE_MASTER_ID = '{}' AND IS_MAIN_ENTITY_FLAG = 0 GROUP BY PERSON_MASTER_ID,RULE_NO,ENTITY_TYPE  UNION  SELECT ORGANIZATION_MASTER_ID AS ENTITY_MASTER_ID,B.ENTITY_TYPE,STRING_AGG(RULE_VALUE,',') WITHIN GROUP (ORDER BY RULE_NO)   AS RULE_VALUE FROM FINCORE_DB.FINCORE_ANALYTICS.RISK_ORGANIZATION_SUMMARY_STG_RULES A INNER JOIN FINCORE_DB.FINCORE.CASE_ENTITY B ON A.ORGANIZATION_MASTER_ID = B.ENTITY_MASTER_ID WHERE RULE_NO IN ('O001001000', 'O001003000') AND ENTITY_TYPE = 'O' AND CASE_MASTER_ID = '{}' AND IS_MAIN_ENTITY_FLAG = 0  GROUP BY ORGANIZATION_MASTER_ID,RULE_NO,ENTITY_TYPE ) SELECT ENTITY_MASTER_ID,ENTITY_TYPE,RULE_VALUE FROM CTE'''.format(cmid,cmid),conn)
        other_entity_watchlist = other_entity_watchlist['RULE_VALUE'].to_list()


        inherent_risk_main = pd.read_sql_query('''with cte as( SELECT PERSON_MASTER_ID,RULE_NO,RISK_SCORE_STATIC,RULE_VALUE FROM FINCORE_DB.FINCORE_ANALYTICS.RISK_PERSON_SUMMARY_STG_RULES WHERE RISK_SCORE_STATIC IS NOT NULL  UNION   SELECT A.PERSON_MASTER_ID,A.RULE_NO,THRESHOLD_FLG AS RISK_SCORE_STATIC,RULE_VALUE   FROM FINCORE_DB.FINCORE_ANALYTICS.RISK_PERSON_ANALYTICAL_RULES  A INNER JOIN FINCORE_DB.FINCORE_ANALYTICS.RISK_PERSON_SUMMARY_STG_RULES B ON A.PERSON_MASTER_ID  = B.PERSON_MASTER_ID AND A.RULE_NO = B.RULE_NO WHERE THRESHOLD_FLG IS NOT NULL ), cte1 as( SELECT ORGANIZATION_MASTER_ID,RULE_NO,RISK_SCORE_STATIC,RULE_VALUE FROM FINCORE_DB.FINCORE_ANALYTICS.RISK_ORGANIZATION_SUMMARY_STG_RULES WHERE RISK_SCORE_STATIC IS NOT NULL UNION  SELECT A.ORGANIZATION_MASTER_ID,A.RULE_NO,THRESHOLD_FLG AS RISK_SCORE_STATIC,RULE_VALUE   FROM FINCORE_DB.FINCORE_ANALYTICS.RISK_ORGANIZATION_ANALYTICAL_RULES  A INNER JOIN FINCORE_DB.FINCORE_ANALYTICS.RISK_ORGANIZATION_SUMMARY_STG_RULES B ON A.ORGANIZATION_MASTER_ID  = B.ORGANIZATION_MASTER_ID AND A.RULE_NO = B.RULE_NO WHERE THRESHOLD_FLG IS NOT NULL  ) select DISTINCT Case_Master_Id,Entity_Master_Id,Entity_Type,RULE_DESC,RULE_VALUE   from  FINCORE_DB.Fincore.Case_Entity CE  left join CTE ON CE.Entity_Master_Id = CTE.PERSON_MASTER_ID INNER JOIN FINCORE_DB.Fincore_Analytics.RISK_LKP_RULES RLR ON RLR.RULE_NO = cte.RULE_NO  WHERE case_master_id = '{}' AND CE.Entity_Type = 'I' AND  risk_score_static is not null and RISK_SCORE_STATIC <> 0   and RISK_TYPE like '% - Inherent Risk%' and CE.Is_Main_Entity_Flag = 1 UNION ALL   select DISTINCT Case_Master_Id,Entity_Master_Id,Entity_Type,RULE_DESC,RULE_VALUE  from FINCORE_DB.Fincore.Case_Entity CE  left join CTE1 ON CE.Entity_Master_Id = CTE1.ORGANIZATION_MASTER_ID   INNER JOIN FINCORE_DB.Fincore_Analytics.RISK_LKP_RULES RLR ON RLR.RULE_NO = cte1.RULE_NO  WHERE CE.Entity_Type =  'O' AND  case_master_id = '{}' AND risk_score_static is not null and RISK_SCORE_STATIC <> 0  and RISK_TYPE like '% - Inherent Risk%' and CE.Is_Main_Entity_Flag = 1'''.format(cmid,cmid),conn)
        z = inherent_risk_main[['RULE_DESC','RULE_VALUE']]
        maximum = z.groupby(['RULE_DESC'])['RULE_VALUE'].max()
        maximum = pd.DataFrame(maximum,columns = ['RULE_DESC_temp','RULE_VALUE'] )
        maximum.reset_index(inplace = True)
        maximum = maximum[['RULE_DESC','RULE_VALUE']]
        maximum['XYZ'] = '-'
        inherent_risk_main = maximum[['RULE_DESC','XYZ','RULE_VALUE']].values.tolist()
        inherent_risk_main = str(inherent_risk_main)
        inherent_risk_main = inherent_risk_main.replace('\',','')
        inherent_risk_main = inherent_risk_main.replace('\'','')
        inherent_risk_main = inherent_risk_main.replace('\"','')
        inherent_risk_main = inherent_risk_main.replace('[','')
        inherent_risk_main = inherent_risk_main.replace(']','')
        
        
        

        inherent_risk_other = pd.read_sql_query(''' with cte as( SELECT PERSON_MASTER_ID,RULE_NO,RISK_SCORE_STATIC,RULE_VALUE FROM FINCORE_DB.FINCORE_ANALYTICS.RISK_PERSON_SUMMARY_STG_RULES WHERE RISK_SCORE_STATIC IS NOT NULL UNION  SELECT A.PERSON_MASTER_ID,A.RULE_NO,THRESHOLD_FLG AS RISK_SCORE_STATIC,RULE_VALUE   FROM FINCORE_DB.FINCORE_ANALYTICS.RISK_PERSON_ANALYTICAL_RULES  A INNER JOIN FINCORE_DB.FINCORE_ANALYTICS.RISK_PERSON_SUMMARY_STG_RULES B ON A.PERSON_MASTER_ID  = B.PERSON_MASTER_ID AND A.RULE_NO = B.RULE_NO WHERE THRESHOLD_FLG IS NOT NULL ), cte1 as( SELECT ORGANIZATION_MASTER_ID,RULE_NO,RISK_SCORE_STATIC,RULE_VALUE FROM FINCORE_DB.FINCORE_ANALYTICS.RISK_ORGANIZATION_SUMMARY_STG_RULES WHERE RISK_SCORE_STATIC IS NOT NULL UNION  SELECT A.ORGANIZATION_MASTER_ID,A.RULE_NO,THRESHOLD_FLG AS RISK_SCORE_STATIC,RULE_VALUE   FROM FINCORE_DB.FINCORE_ANALYTICS.RISK_ORGANIZATION_ANALYTICAL_RULES  A INNER JOIN FINCORE_DB.FINCORE_ANALYTICS.RISK_ORGANIZATION_SUMMARY_STG_RULES B ON A.ORGANIZATION_MASTER_ID  = B.ORGANIZATION_MASTER_ID AND A.RULE_NO = B.RULE_NO WHERE THRESHOLD_FLG IS NOT NULL ) select DISTINCT Case_Master_Id,Entity_Master_Id,Entity_Type,RULE_DESC,RULE_VALUE   from  FINCORE_DB.Fincore.Case_Entity CE left join CTE ON CE.Entity_Master_Id = CTE.PERSON_MASTER_ID INNER JOIN FINCORE_DB.Fincore_Analytics.RISK_LKP_RULES RLR ON RLR.RULE_NO = cte.RULE_NO  WHERE case_master_id = '{}' AND CE.Entity_Type = 'I' AND  risk_score_static is not null and RISK_SCORE_STATIC <> 0  and RISK_TYPE like '% - Inherent Risk%' and CE.Is_Main_Entity_Flag = 0  UNION ALL   select DISTINCT Case_Master_Id,Entity_Master_Id,Entity_Type,RULE_DESC,RULE_VALUE   from FINCORE_DB.Fincore.Case_Entity CE left join CTE1 ON CE.Entity_Master_Id = CTE1.ORGANIZATION_MASTER_ID  INNER JOIN FINCORE_DB.Fincore_Analytics.RISK_LKP_RULES RLR ON RLR.RULE_NO = cte1.RULE_NO  WHERE CE.Entity_Type =  'O' AND  case_master_id = '{}' AND risk_score_static is not null and RISK_SCORE_STATIC <> 0   and RISK_TYPE like '% - Inherent Risk%' and CE.Is_Main_Entity_Flag = 0 '''.format(cmid,cmid),conn)
        z = inherent_risk_other[['RULE_DESC','RULE_VALUE']]
        maximum = z.groupby(['RULE_DESC'])['RULE_VALUE'].max()
        maximum = pd.DataFrame(maximum,columns = ['RULE_DESC_temp','RULE_VALUE'] )
        maximum.reset_index(inplace = True)
        maximum = maximum[['RULE_DESC','RULE_VALUE']]
        maximum['XYZ'] = '-'
        inherent_risk_other = maximum[['RULE_DESC','XYZ','RULE_VALUE']].values.tolist()
        inherent_risk_other = str(inherent_risk_other)
        inherent_risk_other = inherent_risk_other.replace('\',','')
        inherent_risk_other = inherent_risk_other.replace('\'','')
        inherent_risk_other = inherent_risk_other.replace('\"','')
        inherent_risk_other = inherent_risk_other.replace('[','')
        inherent_risk_other = inherent_risk_other.replace(']','')

        
        
        
        
        
        
        

        all_others_df_main = pd.read_sql_query('''  with cte as( SELECT PERSON_MASTER_ID,RULE_NO,RISK_SCORE_STATIC,RULE_VALUE FROM FINCORE_DB.FINCORE_ANALYTICS.RISK_PERSON_SUMMARY_STG_RULES WHERE RISK_SCORE_STATIC IS NOT NULL UNION  SELECT A.PERSON_MASTER_ID,A.RULE_NO,THRESHOLD_FLG AS RISK_SCORE_STATIC,RULE_VALUE   FROM FINCORE_DB.FINCORE_ANALYTICS.RISK_PERSON_ANALYTICAL_RULES  A INNER JOIN FINCORE_DB.FINCORE_ANALYTICS.RISK_PERSON_SUMMARY_STG_RULES B ON A.PERSON_MASTER_ID  = B.PERSON_MASTER_ID AND A.RULE_NO = B.RULE_NO WHERE THRESHOLD_FLG IS NOT NULL ), cte1 as( SELECT ORGANIZATION_MASTER_ID,RULE_NO,RISK_SCORE_STATIC,RULE_VALUE FROM FINCORE_DB.FINCORE_ANALYTICS.RISK_ORGANIZATION_SUMMARY_STG_RULES WHERE RISK_SCORE_STATIC IS NOT NULL UNION  SELECT A.ORGANIZATION_MASTER_ID,A.RULE_NO,THRESHOLD_FLG AS RISK_SCORE_STATIC,RULE_VALUE   FROM FINCORE_DB.FINCORE_ANALYTICS.RISK_ORGANIZATION_ANALYTICAL_RULES  A INNER JOIN FINCORE_DB.FINCORE_ANALYTICS.RISK_ORGANIZATION_SUMMARY_STG_RULES B ON A.ORGANIZATION_MASTER_ID  = B.ORGANIZATION_MASTER_ID AND A.RULE_NO = B.RULE_NO WHERE THRESHOLD_FLG IS NOT NULL )   select DISTINCT Case_Master_Id,Entity_Master_Id,Entity_Type,RULE_DESC,RULE_VALUE  from FINCORE_DB.Fincore.Case_Entity CE   left join CTE ON CE.Entity_Master_Id = CTE.PERSON_MASTER_ID   INNER JOIN FINCORE_DB.Fincore_Analytics.RISK_LKP_RULES RLR ON RLR.RULE_NO = cte.RULE_NO  WHERE case_master_id = '{}' AND CE.Entity_Type = 'I' AND  risk_score_static is not null and RISK_SCORE_STATIC <> 0   and  (RISK_TYPE not like '%- Inherent Risk%' and RISK_TYPE <> 'Auto High Rules ' )  and CE.Is_Main_Entity_Flag = 1  UNION ALL   select DISTINCT Case_Master_Id,Entity_Master_Id,Entity_Type,RULE_DESC,RULE_VALUE  from FINCORE_DB.Fincore.Case_Entity CE left join CTE1 ON CE.Entity_Master_Id = CTE1.ORGANIZATION_MASTER_ID  INNER JOIN FINCORE_DB.Fincore_Analytics.RISK_LKP_RULES RLR ON RLR.RULE_NO = cte1.RULE_NO  WHERE CE.Entity_Type =  'O' AND  case_master_id = '{}' AND risk_score_static is not null and   RISK_SCORE_STATIC <> 0   and    (RISK_TYPE not like '%- Inherent Risk%' and RISK_TYPE <> 'Auto High Rules ' )  and CE.Is_Main_Entity_Flag = 1'''.format(cmid,cmid),conn)
        z = all_others_df_main[['RULE_DESC','RULE_VALUE']]
        maximum = z.groupby(['RULE_DESC'])['RULE_VALUE'].max()
        maximum = pd.DataFrame(maximum,columns = ['RULE_DESC_temp','RULE_VALUE'] )
        maximum.reset_index(inplace = True)
        maximum = maximum[['RULE_DESC','RULE_VALUE']]
        maximum['XYZ'] = '-'
        all_others_df_main = maximum[['RULE_DESC','XYZ','RULE_VALUE']].values.tolist() 
        all_others_df_main = str(all_others_df_main)
        all_others_df_main = all_others_df_main.replace('\',','')
        all_others_df_main = all_others_df_main.replace('\'','')
        all_others_df_main = all_others_df_main.replace('\"','')
        all_others_df_main = all_others_df_main.replace('[','')
        all_others_df_main = all_others_df_main.replace(']','')
        
    
        all_others_df_other = pd.read_sql_query(''' with cte as( SELECT PERSON_MASTER_ID,RULE_NO,RISK_SCORE_STATIC,RULE_VALUE FROM FINCORE_DB.FINCORE_ANALYTICS.RISK_PERSON_SUMMARY_STG_RULES WHERE RISK_SCORE_STATIC IS NOT NULL  UNION   SELECT A.PERSON_MASTER_ID,A.RULE_NO,THRESHOLD_FLG AS RISK_SCORE_STATIC,RULE_VALUE   FROM FINCORE_DB.FINCORE_ANALYTICS.RISK_PERSON_ANALYTICAL_RULES  A INNER JOIN FINCORE_DB.FINCORE_ANALYTICS.RISK_PERSON_SUMMARY_STG_RULES B ON A.PERSON_MASTER_ID  = B.PERSON_MASTER_ID AND A.RULE_NO = B.RULE_NO WHERE THRESHOLD_FLG IS NOT NULL ), cte1 as( SELECT ORGANIZATION_MASTER_ID , RULE_NO ,RISK_SCORE_STATIC ,RULE_VALUE FROM FINCORE_DB.FINCORE_ANALYTICS.RISK_ORGANIZATION_SUMMARY_STG_RULES WHERE RISK_SCORE_STATIC IS NOT NULL  UNION   SELECT A.ORGANIZATION_MASTER_ID,A.RULE_NO,THRESHOLD_FLG AS RISK_SCORE_STATIC,RULE_VALUE   FROM FINCORE_DB.FINCORE_ANALYTICS.RISK_ORGANIZATION_ANALYTICAL_RULES  A INNER JOIN FINCORE_DB.FINCORE_ANALYTICS.RISK_ORGANIZATION_SUMMARY_STG_RULES B ON A.ORGANIZATION_MASTER_ID  = B.ORGANIZATION_MASTER_ID AND A.RULE_NO = B.RULE_NO WHERE THRESHOLD_FLG IS NOT NULL  ) select DISTINCT Case_Master_Id,Entity_Master_Id,Entity_Type,RULE_DESC,RULE_VALUE  from  FINCORE_DB.Fincore.Case_Entity CE  left join CTE ON CE.Entity_Master_Id = CTE.PERSON_MASTER_ID  INNER JOIN FINCORE_DB.Fincore_Analytics.RISK_LKP_RULES RLR ON RLR.RULE_NO = cte.RULE_NO  WHERE case_master_id = '{}' AND CE.Entity_Type = 'I' AND  risk_score_static is not null and RISK_SCORE_STATIC <> 0   and   (RISK_TYPE not like '%- Inherent Risk%' and RISK_TYPE <> 'Auto High Rules ' )  and CE.Is_Main_Entity_Flag = 0  UNION ALL   select DISTINCT Case_Master_Id, Entity_Master_Id, Entity_Type, RULE_DESC, RULE_VALUE    from FINCORE_DB.Fincore.Case_Entity CE left join CTE1 ON CE.Entity_Master_Id = CTE1.ORGANIZATION_MASTER_ID  INNER JOIN FINCORE_DB.Fincore_Analytics.RISK_LKP_RULES RLR ON RLR.RULE_NO = cte1.RULE_NO  WHERE CE.Entity_Type =  'O' AND  case_master_id = '{}' AND risk_score_static is not null and   RISK_SCORE_STATIC <> 0   and   (RISK_TYPE not like '%- Inherent Risk%' and RISK_TYPE <> 'Auto High Rules ' )  and CE.Is_Main_Entity_Flag = 0 '''.format(cmid,cmid),conn)
        z = all_others_df_other[['RULE_DESC','RULE_VALUE']]
        maximum = z.groupby(['RULE_DESC'])['RULE_VALUE'].max()
        maximum = pd.DataFrame(maximum,columns = ['RULE_DESC_temp','RULE_VALUE'] )
        maximum.reset_index(inplace = True)
        maximum = maximum[['RULE_DESC','RULE_VALUE']]
        maximum['XYZ'] = '-'
        all_others_df_other = maximum[['RULE_DESC','XYZ','RULE_VALUE']].values.tolist()   
        all_others_df_other = str(all_others_df_other)
        all_others_df_other = all_others_df_other.replace('\',','')
        all_others_df_other = all_others_df_other.replace('\'','')
        all_others_df_other = all_others_df_other.replace('\"','')
        all_others_df_other = all_others_df_other.replace('[','')
        all_others_df_other = all_others_df_other.replace(']','')

        s = '<html>'
        if RISK_SCORE < 6:
            s = s+'<b>The Main entity associated with this case has a low Risk.</b><ul> '

        if (len(main_entity) > 0):
            s = s+ '<br><b>The Main Entity Risk Factors</b></br><ul>'
            if (auto_df_main == []):

                if (len(main_entity) == 1):
                    if (inherent_risk_main == []):
                        pass
                    else:
                        s = s + "<li> The Main entity has  <b>" + str(inherent_risk_main) +" </b>.</li>"
                    if (all_others_df_main == []):
                        pass
                    else:
                        s = s + "<li>The Main entity has <b>" + str(all_others_df_main) +"</b>.</li>"

                else:
                    if(inherent_risk_main == []):
                        pass
                    else:
                        s = s + "<li> The Main entities have" + str(inherent_risk_main) +".</li>"

                    if (all_others_df_main == []):
                        pass  
                    else:
                        s = s + "<li> The Main entities have <b>" + str(all_others_df_main) +"</b>.</li>"
                        
                s = s+ '</ul>'            
        



            else:
                if (len(main_entity) == 1):
                    s = s+"<li> The main entity has <b>"+ str(auto_df_main) + " </b>and is reported on the <b>" + str(main_entity_watchlist)+"</b>watchlist.</li>"

                elif(len(main_entity) > 1):
                    s = s+"<li> The main entities have <b>"+ str(auto_df_main) + "</b>and are reported on the <b>" + str(main_entity_watchlist)+"</b>watchlist</li>."


        else:
            s = s+"</ul> There is no main entity"



        if (len(other_entity) > 0):
            
            s = s+ '<br></br><b>The Other Entity Risk Factors</b><ul>'
            if (auto_df_other == []):

                if (len(other_entity) == 1):
                    if (inherent_risk_other == []):
                        pass
                    else:
                        s = s + "<li> The Other case entity has <b>" + str(inherent_risk_other) +"</b>.</li>"

                    if (all_others_df_other == []):
                        pass
                    else:
                        s = s + "<li> The Other case entity has <b>" + str(all_others_df_other) +"</b>.</li>"

                else:
                    if(inherent_risk_other == []):
                        pass
                    else:
                        s = s + "<li> The Other case entities have <b>" + str(inherent_risk_other) +"</b>.</li>"
                    if (all_others_df_other == []):
                        pass
                    else:
                        s = s + "<li> The Other case entities have <b> " + str(all_others_df_other) +"</b>.</li>"


            else:
                if (len(other_entity) == 1):
                    s = s+"<li> The Other case  entity has <b>"+ str(auto_df_other) + " </b>and is reported on the <b>" + str(other_entity_watchlist)+"</b>watchlist.</li>"


                elif(len(other_entity) > 1):
                    s = s+"<li> The Other case  entities have <b>"+ str(auto_df_other) + "</b>and are reported on the <b>" + str(other_entity_watchlist)+"</b>watchlist.</li>"

            s = s+ '</ul>'            


        else:
             s = s+"<br></ul> There is no other case entity"








        tags = pd.read_sql_query('''SELECT DISTINCT CASE_MASTER_ID,TAG as TAG_NAME FROM
(
SELECT CASE_MASTER_ID,TAG,
RANK() OVER( PARTITION BY CASE_MASTER_ID ORDER BY TEMP.RISK_SCORE DESC) AS RANK 
FROM 
(
SELECT  DISTINCT CASE_MASTER_ID,A.REPORTID,E.TAG_TYPE,
TAG_RISK.TAG,
TAG_RISK.RISK_SCORE
FROM FINCORE_DB.FINCORE.CASE_MASTER CM WITH(NOLOCK)
INNER JOIN  FINCORE_DB.FINCORE_ANALYTICS.GOS_PROCESSED A  WITH(NOLOCK) ON CM.REPORT_ID = A.REPORTID
INNER JOIN FINCORE_DB.FINCORE_ANALYTICS.RISK_GOS_TAG_LIST E WITH(NOLOCK) ON   A.REPORTID=E.REPORT_ID 
INNER JOIN FINCORE_BRIDGE_DB.FINCORE_BRIDGE.FINGATE_GOS FG WITH(NOLOCK) 
ON  A.REPORTID=FG.REPORTID
INNER JOIN
(
SELECT ID_,NAME AS TAG ,RISK_SCORE AS RISK_SCORE FROM DATAHUB_DB.FIUMETAHUB.[FINNET_SUSCATEGORY]AS FNSC WITH (NOLOCK)
)TAG_RISK
ON CAST(TAG_RISK.ID_ AS NVARCHAR)  = CAST(FG.GOSTAG1  AS NVARCHAR) 
WHERE TAG_RISK.RISK_SCORE IS NOT NULL  AND E.TAG_TYPE='SUSPICION DUE TO'


UNION 

SELECT  DISTINCT CASE_MASTER_ID,A.REPORTID,E.TAG_TYPE,
TAG_RISK.TAG,
TAG_RISK.RISK_SCORE
FROM FINCORE_DB.FINCORE.CASE_MASTER CM WITH(NOLOCK)
INNER JOIN  FINCORE_DB.FINCORE_ANALYTICS.GOS_PROCESSED A  WITH(NOLOCK) ON CM.REPORT_ID = A.REPORTID
INNER JOIN FINCORE_DB.FINCORE_ANALYTICS.RISK_GOS_TAG_LIST E  WITH(NOLOCK) ON   A.REPORTID=E.REPORT_ID 
INNER JOIN FINCORE_BRIDGE_DB.FINCORE_BRIDGE.FINGATE_GOS FG WITH(NOLOCK) 
ON  A.REPORTID=FG.REPORTID
INNER JOIN
(
SELECT ID_,SOURCE AS TAG,RISK_SCORE AS RISK_SCORE FROM  DATAHUB_DB.FIUMETAHUB.FINNET_ALERTSOURCE AS FNAS WITH (NOLOCK)
)TAG_RISK
ON  CAST(TAG_RISK.ID_  AS NVARCHAR)  = CAST(FG.GOSTAG3  AS NVARCHAR)
WHERE TAG_RISK.RISK_SCORE IS NOT NULL AND E.TAG_TYPE='SOURCE OF ALERT'

UNION 

SELECT  DISTINCT CASE_MASTER_ID,A.REPORTID,E.TAG_TYPE,
TAG_RISK.TAG,
TAG_RISK.RISK_SCORE
FROM FINCORE_DB.FINCORE.CASE_MASTER CM WITH(NOLOCK)
INNER JOIN  FINCORE_DB.FINCORE_ANALYTICS.GOS_PROCESSED A WITH(NOLOCK) ON CM.REPORT_ID = A.REPORTID
INNER JOIN FINCORE_DB.FINCORE_ANALYTICS.RISK_GOS_TAG_LIST E WITH(NOLOCK) ON   A.REPORTID=E.REPORT_ID 
INNER JOIN FINCORE_BRIDGE_DB.FINCORE_BRIDGE.FINGATE_GOS FG WITH(NOLOCK) 
ON  A.REPORTID=FG.REPORTID
INNER JOIN
(
SELECT ID_,TAGVALUE AS TAG,RISK_SCORE AS RISK_SCORE FROM  DATAHUB_DB.FIUMETAHUB.FINNET_OFFENSETYPE AS FNAS WITH (NOLOCK)

)TAG_RISK
ON CAST(TAG_RISK.ID_  AS NVARCHAR)  = CAST(FG.GOSTAG4  AS NVARCHAR) 
WHERE TAG_RISK.RISK_SCORE IS NOT NULL  AND E.TAG_TYPE='OFFENCE SUSPECTED'
)TEMP
)TEMP1
WHERE TEMP1.RANK <6 AND CASE_MASTER_ID = '{}' '''.format(cmid),conn)
        tags = tags['TAG_NAME'].values.tolist()
        tags = str(tags)
        tags = tags.replace('\'','')
        tags = tags.replace('[','')  
        tags = tags.replace(']','')

        if (len(tags) == 0):
            pass
        else:
#             s = s +'<br></ol>Multiple, invalid or wrong PAN/DIN/ DPIN/TAN/IEC number(s) has been reported on <b>'+ str(multiple_pan_din) +'</b>entities.'
            s = s +'<br></ul><li>There is a suspicion of <b> '+ tags +'</b>.'




        transaction_rules = pd.read_sql_query('''  SELECT distinct C.RULE_DESC ,RULE_VALUE FROM FINCORE_DB.FINCORE.CASE_MASTER A WITH (NOLOCK) INNER JOIN FINCORE_DB.FINCORE_ANALYTICS.RISK_TRANSACTION_RULES B WITH (NOLOCK) ON A.REPORT_ID = B.REPORT_ID INNER JOIN  FINCORE_DB.FINCORE_ANALYTICS.RISK_LKP_RULES C WITH (NOLOCK) ON B.RULE_NO = C.RULE_NO WHERE  RISK_SCORE_STATIC > 0 AND  A.CASE_MASTER_ID = '{}' '''.format(cmid),conn)        
        transaction_rules['XYZ'] = '-'
        transaction_rules = transaction_rules[['RULE_DESC','XYZ','RULE_VALUE']].values.tolist()
        transaction_rules = str(transaction_rules)
        transaction_rules = transaction_rules.replace('\',','')
        transaction_rules = transaction_rules.replace('\'','')
        transaction_rules = transaction_rules.replace('\"','')
        transaction_rules = transaction_rules.replace('[','')
        transaction_rules = transaction_rules.replace(']','')
        transaction_rules = transaction_rules.replace('\\t',' ')

        if (len(transaction_rules) == 0):
            pass
        else:
            s = s+ "</ul><li> The transaction rules that are flagged are <b>"+ transaction_rules + "</b>.</li>"




        CM = pd.read_sql_query("select * from Fincore.Case_Entity_Reports where Case_Master_Id = '{}' ".format(cmid),conn)
        RTSS = pd.read_sql_query("select * from fincore_db.fincore_analytics.risk_transaction_summary_stg_TRAN",conn)
        FP = pd.read_sql_query("select pinCodeNumber,risk_score from DATAHUB_DB.FIUMetaHub.finnet_pincode",conn)
        CM_RTSS = CM.merge(RTSS,left_on = 'Report_Id',right_on = 'REPORT_ID' ,how = 'inner' )
        CM_RTSS_FP = CM_RTSS.merge(FP,left_on = 'LOCATION_PINCODE',right_on = 'pinCodeNumber',how = 'inner')
        CM_RTSS_FP = CM_RTSS_FP[['Case_Master_Id','LOCATION_PINCODE','risk_score']]
        CM_RTSS_FP.drop_duplicates(inplace = True)
        CM_RTSS_FP = CM_RTSS_FP[CM_RTSS_FP['risk_score'] > 7]
        CM_RTSS_FP['RANK'] = CM_RTSS_FP.sort_values(['risk_score'],ascending = False).groupby(['Case_Master_Id']).cumcount()+1
        CM_RTSS_FP['LOCATION_PINCODE'] = CM_RTSS_FP['LOCATION_PINCODE'].astype('int64') 
        CM_RTSS_FP =  CM_RTSS_FP.query('RANK <= 3')
        pincode_top_3 = CM_RTSS_FP['LOCATION_PINCODE'].to_list()
        pincode_top_3 = str(pincode_top_3)
        pincode_top_3 = pincode_top_3.replace('[','')
        pincode_top_3 = pincode_top_3.replace(']','',)

        if (len(pincode_top_3) == 0):
            pass
        else:
            s  = s + "</ul><li>High-risk locations of the transactions involved are <b>" + str(pincode_top_3) + ".</b>"



        no_of_sanctioned_id = pd.read_sql_query('''SELECT ENTITY_MASTER_ID,ENTITY_TYPE FROM FINCORE_DB.FINCORE.CASE_ENTITY A INNER JOIN FINCORE_DB.FINCORE_ANALYTICS.RISK_PERSON_SUMMARY_STG_RULES B ON A.ENTITY_MASTER_ID = B.PERSON_MASTER_ID INNER JOIN FINCORE_DB.FINCORE_ANALYTICS.RISK_LKP_RULES C ON B.RULE_NO = C.RULE_NO WHERE ENTITY_TYPE = 'I' AND Case_Master_Id = '{}' AND C.RULE_NO = 'P001001000' UNION  SELECT ENTITY_MASTER_ID,ENTITY_TYPE FROM FINCORE_DB.FINCORE.CASE_ENTITY A INNER JOIN FINCORE_DB.FINCORE_ANALYTICS.RISK_ORGANIZATION_SUMMARY_STG_RULES B ON A.ENTITY_MASTER_ID = B.ORGANIZATION_MASTER_ID INNER JOIN FINCORE_DB.FINCORE_ANALYTICS.RISK_LKP_RULES C ON B.RULE_NO = C.RULE_NO WHERE ENTITY_TYPE = 'O' AND C.RULE_NO = 'O001001000' AND Case_Master_Id = '{}' '''.format(cmid,cmid),conn)
        no_of_sanctioned_id  = no_of_sanctioned_id[['ENTITY_MASTER_ID','ENTITY_TYPE']].values.tolist()

        if (len(no_of_sanctioned_id) == 0):
            pass
        else:
#             s = s+ "<br></ol>This case has <b>"+ str(no_of_sanctioned_id) + "</b> entities on the FIU or LEA watchlist."
            s = s+ "</ul><li>This case has <b>"+ str(len(no_of_sanctioned_id)) + "</b> entities on the FIU or LEA watchlist.</li>"








        lea_adhoc = pd.read_sql_query('''SELECT ENTITY_MASTER_ID,ENTITY_TYPE FROM FINCORE_DB.FINCORE.CASE_ENTITY A INNER JOIN FINCORE_DB.FINCORE_ANALYTICS.RISK_PERSON_SUMMARY_STG_RULES B ON A.ENTITY_MASTER_ID = B.PERSON_MASTER_ID INNER JOIN FINCORE_DB.FINCORE_ANALYTICS.RISK_LKP_RULES C ON B.RULE_NO = C.RULE_NO WHERE ENTITY_TYPE = 'I' AND Case_Master_Id = '{}' AND C.RULE_NO = 'P001002000' UNION  SELECT ENTITY_MASTER_ID,ENTITY_TYPE FROM FINCORE_DB.FINCORE.CASE_ENTITY A INNER JOIN FINCORE_DB.FINCORE_ANALYTICS.RISK_ORGANIZATION_SUMMARY_STG_RULES B ON A.ENTITY_MASTER_ID = B.ORGANIZATION_MASTER_ID INNER JOIN FINCORE_DB.FINCORE_ANALYTICS.RISK_LKP_RULES C ON B.RULE_NO = C.RULE_NO WHERE ENTITY_TYPE = 'O' AND C.RULE_NO = 'O001002000' AND Case_Master_Id = '{}' '''.format(cmid,cmid),conn)
        lea_adhoc  = lea_adhoc[['ENTITY_MASTER_ID','ENTITY_TYPE']].values.tolist()



        if (len(lea_adhoc) == 0):
            pass
        else:
#             s = s+ "<br></ol>LEA Adhoc request has been previously raised on <b> "+ str(lea_adhoc) + "</b> entities."
            s = s+ "</ul><li>LEA Adhoc request has been previously raised on <b> "+ str(len(lea_adhoc)) + "</b> entities.</li>"


        international_sanction_list = pd.read_sql_query('''SELECT ENTITY_MASTER_ID,ENTITY_TYPE FROM FINCORE_DB.FINCORE.CASE_ENTITY A INNER JOIN FINCORE_DB.FINCORE_ANALYTICS.RISK_PERSON_SUMMARY_STG_RULES B ON A.ENTITY_MASTER_ID = B.PERSON_MASTER_ID INNER JOIN FINCORE_DB.FINCORE_ANALYTICS.RISK_LKP_RULES C ON B.RULE_NO = C.RULE_NO WHERE ENTITY_TYPE = 'I' AND Case_Master_Id = '{}' AND C.RULE_NO = 'P001003000' UNION  SELECT ENTITY_MASTER_ID,ENTITY_TYPE FROM FINCORE_DB.FINCORE.CASE_ENTITY A INNER JOIN FINCORE_DB.FINCORE_ANALYTICS.RISK_ORGANIZATION_SUMMARY_STG_RULES B ON A.ENTITY_MASTER_ID = B.ORGANIZATION_MASTER_ID INNER JOIN FINCORE_DB.FINCORE_ANALYTICS.RISK_LKP_RULES C ON B.RULE_NO = C.RULE_NO WHERE ENTITY_TYPE = 'O' AND C.RULE_NO = 'O001003000' AND Case_Master_Id = '{}' '''.format(cmid,cmid),conn)
        international_sanction_list  = international_sanction_list[['ENTITY_MASTER_ID','ENTITY_TYPE']].values.tolist()



        if (len(international_sanction_list) == 0):
            pass
        else:
#             s = s +'<br><b></ol>'+ str(international_sanction_list) +'</b>entities are flagged in the international sanctions lists.'
            s = s +'</ul><b><li>'+ str(len(international_sanction_list)) +'</b>entities are flagged in the international sanctions lists.</li>'




        adverse_media_screening = pd.read_sql_query('''SELECT ENTITY_MASTER_ID,ENTITY_TYPE FROM FINCORE_DB.FINCORE.CASE_ENTITY A INNER JOIN FINCORE_DB.FINCORE_ANALYTICS.RISK_PERSON_SUMMARY_STG_RULES B ON A.ENTITY_MASTER_ID = B.PERSON_MASTER_ID INNER JOIN FINCORE_DB.FINCORE_ANALYTICS.RISK_LKP_RULES C ON B.RULE_NO = C.RULE_NO WHERE ENTITY_TYPE = 'I' AND Case_Master_Id = '{}' AND C.RULE_NO = 'P001004000' UNION  SELECT ENTITY_MASTER_ID,ENTITY_TYPE FROM FINCORE_DB.FINCORE.CASE_ENTITY A INNER JOIN FINCORE_DB.FINCORE_ANALYTICS.RISK_ORGANIZATION_SUMMARY_STG_RULES B ON A.ENTITY_MASTER_ID = B.ORGANIZATION_MASTER_ID INNER JOIN FINCORE_DB.FINCORE_ANALYTICS.RISK_LKP_RULES C ON B.RULE_NO = C.RULE_NO WHERE ENTITY_TYPE = 'O' AND C.RULE_NO = 'O001004000' AND Case_Master_Id = '{}' '''.format(cmid,cmid),conn)
        adverse_media_screening  = adverse_media_screening[['ENTITY_MASTER_ID','ENTITY_TYPE']].values.tolist()


        if (len(adverse_media_screening) == 0):
            pass
        else:
#             s = s +'<br></ol>Adverse Media Screening reveals auto high-risk tags on <b>'+ str(adverse_media_screening) +'</b>entities.'
            s = s +'<br></ul><li>Adverse Media Screening reveals auto high-risk tags on <b>'+ str(len(adverse_media_screening)) +'</b>entities.'



        high_risk_prof = pd.read_sql_query('''SELECT ENTITY_MASTER_ID,ENTITY_TYPE,RULE_VALUE,RISK_SCORE_STATIC as RISK_SCORE  FROM FINCORE_DB.FINCORE.CASE_ENTITY A INNER JOIN FINCORE_DB.FINCORE_ANALYTICS.RISK_PERSON_SUMMARY_STG_RULES B ON A.ENTITY_MASTER_ID = B.PERSON_MASTER_ID INNER JOIN FINCORE_DB.FINCORE_ANALYTICS.RISK_LKP_RULES C ON B.RULE_NO = C.RULE_NO WHERE ENTITY_TYPE = 'I' AND RISK_SCORE_STATIC > 7 AND Case_Master_Id = '{}' AND C.RULE_NO = 'P003001002' UNION  SELECT ENTITY_MASTER_ID,ENTITY_TYPE,RULE_VALUE,RISK_SCORE_STATIC as RISK_SCORE FROM FINCORE_DB.FINCORE.CASE_ENTITY A INNER JOIN FINCORE_DB.FINCORE_ANALYTICS.RISK_ORGANIZATION_SUMMARY_STG_RULES B ON A.ENTITY_MASTER_ID = B.ORGANIZATION_MASTER_ID INNER JOIN FINCORE_DB.FINCORE_ANALYTICS.RISK_LKP_RULES C ON B.RULE_NO = C.RULE_NO WHERE ENTITY_TYPE = 'O' AND RISK_SCORE_STATIC > 7  AND C.RULE_NO = 'O003001002' AND Case_Master_Id = '{}' '''.format(cmid,cmid),conn)
        high_risk_prof_len  = high_risk_prof[['ENTITY_MASTER_ID','ENTITY_TYPE','RULE_VALUE']].values.tolist()
        high_risk_prof = high_risk_prof['RULE_VALUE'].values.tolist()
        high_risk_prof = str(high_risk_prof)
        high_risk_prof = high_risk_prof.replace('\'','')
        high_risk_prof = high_risk_prof.replace('[','')  
        high_risk_prof = high_risk_prof.replace(']','')

        if (len(high_risk_prof_len) == 0):
            pass
        else:
#             s = s +'<br></ol>Multiple, invalid or wrong PAN/DIN/ DPIN/TAN/IEC number(s) has been reported on <b>'+ str(multiple_pan_din) +'</b>entities.'
            s = s +'<br></ul><b><li>'+ str(len(high_risk_prof_len)) +'</b> are engaged in High Risk businesses such as <b>' + high_risk_prof + ' </b>.'

        multiple_pan_din = pd.read_sql_query('''SELECT ENTITY_MASTER_ID,ENTITY_TYPE FROM FINCORE_DB.FINCORE.CASE_ENTITY A INNER JOIN FINCORE_DB.FINCORE_ANALYTICS.RISK_PERSON_SUMMARY_STG_RULES B ON A.ENTITY_MASTER_ID = B.PERSON_MASTER_ID INNER JOIN FINCORE_DB.FINCORE_ANALYTICS.RISK_LKP_RULES C ON B.RULE_NO = C.RULE_NO WHERE ENTITY_TYPE = 'I' AND Case_Master_Id = '{}' AND C.RULE_NO = 'P001006000' UNION  SELECT ENTITY_MASTER_ID,ENTITY_TYPE FROM FINCORE_DB.FINCORE.CASE_ENTITY A INNER JOIN FINCORE_DB.FINCORE_ANALYTICS.RISK_ORGANIZATION_SUMMARY_STG_RULES B ON A.ENTITY_MASTER_ID = B.ORGANIZATION_MASTER_ID INNER JOIN FINCORE_DB.FINCORE_ANALYTICS.RISK_LKP_RULES C ON B.RULE_NO = C.RULE_NO WHERE ENTITY_TYPE = 'O' AND C.RULE_NO = 'O001006000' AND Case_Master_Id = '{}' '''.format(cmid,cmid),conn)
        multiple_pan_din  = multiple_pan_din[['ENTITY_MASTER_ID','ENTITY_TYPE']].values.tolist()

        if (len(multiple_pan_din) == 0):
            pass
        else:
#             s = s +'<br></ol>Multiple, invalid or wrong PAN/DIN/ DPIN/TAN/IEC number(s) has been reported on <b>'+ str(multiple_pan_din) +'</b>entities.'
            s = s +'<br></ul><li>Multiple, invalid or wrong PAN/DIN/ DPIN/TAN/IEC number(s) has been reported on <b>'+ str(len(multiple_pan_din)) +'</b>entities.</br>'



        serious_fraud = pd.read_sql_query('''SELECT ENTITY_MASTER_ID,ENTITY_TYPE FROM FINCORE_DB.FINCORE.CASE_ENTITY A INNER JOIN FINCORE_DB.FINCORE_ANALYTICS.RISK_PERSON_SUMMARY_STG_RULES B ON A.ENTITY_MASTER_ID = B.PERSON_MASTER_ID INNER JOIN FINCORE_DB.FINCORE_ANALYTICS.RISK_LKP_RULES C ON B.RULE_NO = C.RULE_NO WHERE ENTITY_TYPE = 'I' AND Case_Master_Id = '{}' AND C.RULE_NO = 'P001001000' UNION  SELECT ENTITY_MASTER_ID,ENTITY_TYPE FROM FINCORE_DB.FINCORE.CASE_ENTITY A INNER JOIN FINCORE_DB.FINCORE_ANALYTICS.RISK_ORGANIZATION_SUMMARY_STG_RULES B ON A.ENTITY_MASTER_ID = B.ORGANIZATION_MASTER_ID INNER JOIN FINCORE_DB.FINCORE_ANALYTICS.RISK_LKP_RULES C ON B.RULE_NO = C.RULE_NO WHERE ENTITY_TYPE = 'O' AND C.RULE_NO = 'O001001000' AND Case_Master_Id = '{}' '''.format(cmid,cmid),conn)
        serious_fraud  = serious_fraud[['ENTITY_MASTER_ID','ENTITY_TYPE']].values.tolist()

        if (len(serious_fraud) == 0):
            pass
        else:
#             s = s +'<br></ol>Multiple, invalid or wrong PAN/DIN/ DPIN/TAN/IEC number(s) has been reported on <b>'+ str(multiple_pan_din) +'</b>entities.'
            s = s +'<br><b><li>'+ str(len(serious_fraud)) +  '</b>entities are flagged in the Serious Fraud Investigation Office list <b>' 





        high_risk_report = pd.read_sql_query('''SELECT CASE_ID,COUNT(DISTINCT LINKED_REPORT_ID) AS NO_OF_REPORT_RISK FROM FINCORE_ANALYTICS_V.CASE_DATA_LINKED_REPORT_RISK WHERE LINKED_REPORT_RISK > 7  and case_id = '{}' GROUP BY CASE_ID'''.format(cmid),conn)
        high_risk_report = high_risk_report['NO_OF_REPORT_RISK'].to_list()

        if (len(high_risk_report) == 0):
            pass
        else:
#             s = s +'<br></ol>Multiple, invalid or wrong PAN/DIN/ DPIN/TAN/IEC number(s) has been reported on <b>'+ str(multiple_pan_din) +'</b>entities.'
            s = s +'</ul><b><li> '+ str(high_risk_report[0]) +'</b>  reports linked to the case are High-Risk.'





        s= s+'</html>'

        s = s.replace('\'','')
        s = s.replace('\"','')
        s = s.replace('[','')
        s = s.replace(']','')
        s
    print(s)
    cursor.execute('''Insert into FINCORE_DB.Fincore_Analytics.risk_summary 
        VALUES ('{}','{}',GETDATE(),GETDATE())''' .format(cmid,s))
    conn.commit()

        

        



<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html><b>The Main entity associated with this case has a low Risk.</b><ul> <br><b>The Main Entity Risk Factors</b></br><ul><li> The Main entity has  <b>Type of accounts held - Current Account </b>.</li><li>The Main entity has <b>Ageing of Reports Filed (STR, CTR, CBWTR, CCR, NTR) - 7</b>.</li></ul><br></ul> There is no other case entity</html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There i

<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>


<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>


<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>


<html><b>The Main entity associated with this case has a low Risk.</b><ul> <br><b>The Main Entity Risk Factors</b></br><ul><li> The Main entity has  <b>Type of accounts held - Current Account </b>.</li><li>The Main entity has <b></b>.</li></ul><br></ul> There is no other case entity</html>
<html><b>The Main entity associated with this case has a low Risk.</b><ul> <br><b>The Main Entity Risk Factors</b></br><ul><li> The Main entity has  <b>Type of accounts held - Current Account </b>.</li><li>The Main entity has <b></b>.</li></ul><br></ul> There is no other case entity</html>
<html> There is no main entity. </html>
<html> There is no main entity. </html>
<html><b>The Main entity associated with this case has a low Risk.</b><ul> <br><b>The Main Entity Risk Factors</b></br><ul><li> The Main entity has  <b>Type of accounts held - Current Account </b>.</li><li>The Main entity has <b></b>.</li></ul><br></ul> There is no other case entity</html>
<html> There is no main entity. </html>
<html><

In [15]:

# for cmid in cmids:
# #     print(cmid)
#     s = ''

#     case_entities = pd.read_sql_query("select CASE_MASTER_ID,ENTITY_MASTER_ID,ENTITY_TYPE,IS_MAIN_ENTITY_FLAG from FINCORE_DB.Fincore.Case_Entity where Is_Active_Flag= 1 and Case_Master_Id = '{}'".format(cmid),conn)

#     all_entities = case_entities[['ENTITY_MASTER_ID','ENTITY_TYPE']].values.tolist()

#     main_entity_df =  case_entities[case_entities['Is_Main_Entity_Flag'] == 1][['ENTITY_MASTER_ID','ENTITY_TYPE']]
#     other_entity = case_entities[case_entities['Is_Main_Entity_Flag'] == 0][['ENTITY_MASTER_ID','ENTITY_TYPE']]

#     main_entity = case_entities[case_entities['Is_Main_Entity_Flag'] == 1][['ENTITY_MASTER_ID','ENTITY_TYPE']].values.tolist()
#     other_entity_df = case_entities[case_entities['Is_Main_Entity_Flag'] == 0][['ENTITY_MASTER_ID','ENTITY_TYPE']].values.tolist()
    
# #     main_entity = case_entities[case_entities['Is_Main_Entity_Flag'] == 1]['Entity_Master_Id'].to_list()
# #     other_entity = case_entities[case_entities['Is_Main_Entity_Flag'] == 0]['Entity_Master_Id'].to_list()


    
    
#     if (len(other_entity) > 5):
#         risk_entity_top_5 = pd.read_sql_query("WITH T AS (SELECT B.CASE_MASTER_ID,B.ENTITY_MASTER_ID,B.ENTITY_TYPE,A.RISK_SCORE FROM FINCORE_DB.FINCORE.CASE_ENTITY B LEFT JOIN FINCORE_DB.FINCORE_ANALYTICS.RISK_PERSON A ON A.PERSON_MASTER_ID = B.ENTITY_MASTER_ID WHERE B.ENTITY_TYPE = 'I' AND  CASE_MASTER_ID = '{}' and Is_Main_Entity_Flag = 0 UNION  SELECT B.CASE_MASTER_ID,B.ENTITY_MASTER_ID,B.ENTITY_TYPE,A.RISK_SCORE FROM FINCORE_DB.FINCORE.CASE_ENTITY B LEFT JOIN FINCORE_DB.FINCORE_ANALYTICS.RISK_ORGANIZATION A  ON A.ORGANIZATION_MASTER_ID = B.ENTITY_MASTER_ID WHERE B.ENTITY_TYPE = 'O' AND CASE_MASTER_ID = '{}' and Is_Main_Entity_Flag = 0) SELECT top 5 T.ENTITY_MASTER_ID,T.ENTITY_TYPE,T.RISK_SCORE FROM T order by T.RISK_SCORE".format(cmid,cmid),conn)
#         other_entity = risk_person_top_5[['ENTITY_MASTER_ID','ENTITY_TYPE']].values.tolist()
        



# #     risk_person = pd.read_sql_query("SELECT * FROM FINCORE_ANALYTICS.RISK_PERSON",conn)
#     risk_entity = pd.read_sql_query("WITH T AS (SELECT B.CASE_MASTER_ID,B.ENTITY_MASTER_ID,B.ENTITY_TYPE,A.RISK_SCORE FROM FINCORE_DB.FINCORE.CASE_ENTITY B LEFT JOIN FINCORE_DB.FINCORE_ANALYTICS.RISK_PERSON A ON A.PERSON_MASTER_ID = B.ENTITY_MASTER_ID WHERE B.ENTITY_TYPE = 'I' AND  CASE_MASTER_ID = '{}' and Is_Main_Entity_Flag = 1 UNION  SELECT B.CASE_MASTER_ID,B.ENTITY_MASTER_ID,B.ENTITY_TYPE,A.RISK_SCORE FROM FINCORE_DB.FINCORE.CASE_ENTITY B LEFT JOIN FINCORE_DB.FINCORE_ANALYTICS.RISK_ORGANIZATION A  ON A.ORGANIZATION_MASTER_ID = B.ENTITY_MASTER_ID WHERE B.ENTITY_TYPE = 'O' AND CASE_MASTER_ID = '{}' and Is_Main_Entity_Flag = 1) SELECT  T.ENTITY_MASTER_ID,T.ENTITY_TYPE,T.RISK_SCORE FROM T ".format(cmid,cmid),conn)
    
# #     RISK_SCORE = risk_entity[risk_entity['PERSON_MASTER_ID'].isin(main_entity) ]['RISK_SCORE'].to_list()
#     RISK_SCORE = risk_entity['RISK_SCORE'].to_list()
#     RISK_SCORE  = RISK_SCORE[0]    
#     try:
#         RISK_SCORE = risk_entity['RISK_SCORE'].to_list()
#         RISK_SCORE  = RISK_SCORE[0]      
#     except IndexError:
#         RISK_SCORE = 0
        
    
        
#     if RISK_SCORE < 5:
#         s = s+'<html> The Entity associated with this case has low Risk </html>'
        

    
#     else:
#         risk_person = pd.read_sql_query("SELECT * FROM FINCORE_ANALYTICS.RISK_PERSON",conn)
#         risk_org = pd.read_sql_query("SELECT * FROM FINCORE_ANALYTICS.RISK_ORGANIZATION",conn)
    
#         auto_high_rules = pd.read_sql_query("Select * from fincore_analytics.risk_lkp_rules where RISK_TYPE = 'Auto High Rules ' and IS_ACTIVE_FLAG = 1",conn)
#         auto_high_rules = auto_high_rules['RULE_NO'].to_list()


#         auto_df = []
#         for ind,row in risk_person.iterrows():
#             l = []
#             l = list(row['RISK_RULE_LIST'].split('_'))
#             auto = row['PERSON_MASTER_ID'],'I',rules_triggered(l,auto_high_rules)
#             auto = list(auto)
#             auto_df.append(auto)
                
#         for ind,row in risk_org.iterrows():
#             l = []
#             l = list(row['RISK_RULE_LIST'].split('_'))
#             auto = row['ORGANIZATION_MASTER_ID'],'O',rules_triggered(l,auto_high_rules)
#             auto = list(auto)
#             auto_df.append(auto)

#         auto_df = pd.DataFrame(auto_df,columns = ['ENTITY_MASTER_ID','ENTITY_TYPE','RULES_TRIGGERED'])
#         auto_df['RULES_TRIGGERED']=auto_df['RULES_TRIGGERED'].astype(str)
#         auto_df=auto_df[auto_df['RULES_TRIGGERED']!='[]']

#         auto_df.reset_index(inplace = True)
#         auto_df.drop(columns = ['index'],inplace = True)


#         for ind,row in auto_df.iterrows():
#             auto_df['RULES_TRIGGERED'][ind] = auto_df['RULES_TRIGGERED'][ind].replace('[','')
#             auto_df['RULES_TRIGGERED'][ind] = auto_df['RULES_TRIGGERED'][ind].replace(']','')
#             auto_df['RULES_TRIGGERED'][ind] = auto_df['RULES_TRIGGERED'][ind].replace('\'','')



#         auto_df_main = auto_df.merge(main_entity_df,on = ['ENTITY_MASTER_ID','ENTITY_TYPE'],how = 'inner')
# #         auto_df_main.reset_index(inplace = True)
#         auto_df_main = auto_df_main['RULES_TRIGGERED'].to_list()
#         auto_df_main = str(auto_df_main)
#         auto_df_main = auto_df_main.replace('\'','')
#         auto_df_main = auto_df_main.replace('[','')
#         auto_df_main = auto_df_main.replace(']','')
#         auto_df_main = auto_df_main.replace(' ','')
#         auto_df_main = auto_df_main.split(',')
#         auto_df_main = [*set(auto_df_main)]


#         auto_df_main = rules_df[rules_df['RULE_NO'].isin(auto_df_main)]['RULE_DESC'].to_list()


#         auto_df_other = auto_df.merge(other_entity_df,on = ['ENTITY_MASTER_ID','ENTITY_TYPE'],how = 'inner')
#         auto_df_other.reset_index(inplace = True)
#         auto_df_other = auto_df_other['RULES_TRIGGERED'].to_list()
#         auto_df_other = str(auto_df_other)
#         auto_df_other = auto_df_other.replace('\'','')
#         auto_df_other = auto_df_other.replace('[','')
#         auto_df_other = auto_df_other.replace(']','')
#         auto_df_other = auto_df_other.replace(' ','')
#         auto_df_other = auto_df_other.split(',')
#         auto_df_other = [*set(auto_df_other)]

#         auto_df_other = rules_df[rules_df['RULE_NO'].isin(auto_df_other)]['RULE_DESC'].to_list()

#         watchlist = pd.read_sql_query("SELECT PERSON_MASTER_ID,RULE_NO,STRING_AGG(RULE_VALUE,',') WITHIN GROUP (ORDER BY RULE_NO)  AS RULE_VALUE FROM FINCORE_DB.FINCORE_ANALYTICS.RISK_PERSON_SUMMARY_STG_RULES WHERE RULE_NO IN ('P001001000', 'P001003000')GROUP BY PERSON_MASTER_ID,RULE_NO UNION SELECT ORGANIZATION_MASTER_ID,RULE_NO,STRING_AGG(RULE_VALUE,',') WITHIN GROUP (ORDER BY RULE_NO)  AS RULE_VALUE FROM FINCORE_DB.FINCORE_ANALYTICS.RISK_ORGANIZATION_SUMMARY_STG_RULES WHERE RULE_NO IN ('O001001000', 'O001003000')GROUP BY ORGANIZATION_MASTER_ID,RULE_NO",conn)

#         main_entity_watchlist = watchlist[watchlist['PERSON_MASTER_ID'].isin(main_entity)]
#         main_entity_watchlist.reset_index(inplace = True)

#         main_entity_watchlist = main_entity_watchlist['RULE_NO'].to_list()
#         main_entity_watchlist = str(main_entity_watchlist)
#         main_entity_watchlist = main_entity_watchlist.replace('\'','')
#         main_entity_watchlist = main_entity_watchlist.replace('[','')
#         main_entity_watchlist = main_entity_watchlist.replace(']','')
#         main_entity_watchlist = main_entity_watchlist.replace(' ','')
#         main_entity_watchlist = main_entity_watchlist.split(',')
#         main_entity_watchlist = [*set(main_entity_watchlist)]

#         other_entity_watchlist = watchlist[watchlist['PERSON_MASTER_ID'].isin(other_entity)]
#         other_entity_watchlist.reset_index(inplace = True)
#         other_entity_watchlist = other_entity_watchlist['RULE_NO'].to_list()
#         other_entity_watchlist = str(other_entity_watchlist)
#         other_entity_watchlist = other_entity_watchlist.replace('\'','')
#         other_entity_watchlist = other_entity_watchlist.replace('[','')
#         other_entity_watchlist = other_entity_watchlist.replace(']','')
#         other_entity_watchlist = other_entity_watchlist.replace(' ','')
#         other_entity_watchlist = other_entity_watchlist.split(',')
#         other_entity_watchlist = [*set(other_entity_watchlist)]


#         inherent_risk = pd.read_sql_query("select * from FINCORE_DB.Fincore_Analytics.RISK_LKP_RULES where person_type = 'Individual' and is_active_flag = 1 and RISK_TYPE = 'KYC profile/ Inherent risk based High Risk Indicators - Inherent Risk'",conn)


#         inherent_risk = inherent_risk['RULE_NO'].to_list()



#         inherent_df = []
#         for ind,row in risk_person.iterrows():
#                 l = []
#                 l = list(row['RISK_RULE_LIST'].split('_'))
#                 inherent = row['PERSON_MASTER_ID'],rules_triggered(l,inherent_risk)
#                 inherent = list(inherent)
#                 inherent_df.append(inherent)



#         inherent_df = pd.DataFrame(inherent_df,columns = ['PERSON_MASTER_ID','RULES_TRIGGERED'])
#         inherent_df['RULES_TRIGGERED']=inherent_df['RULES_TRIGGERED'].astype(str)
#         inherent_df=inherent_df[inherent_df['RULES_TRIGGERED']!='[]']
#         inherent_df.reset_index(inplace = True)
#         inherent_df.drop(columns = ['index'],inplace = True)


#         for ind,row in inherent_df.iterrows():
#             inherent_df['RULES_TRIGGERED'][ind] = inherent_df['RULES_TRIGGERED'][ind].replace('[','')
#             inherent_df['RULES_TRIGGERED'][ind] = inherent_df['RULES_TRIGGERED'][ind].replace(']','')
#             inherent_df['RULES_TRIGGERED'][ind]=inherent_df['RULES_TRIGGERED'][ind].replace('\'','')



#         inherent_risk_main = inherent_df[inherent_df['PERSON_MASTER_ID'].isin(main_entity)]
#         inherent_risk_main.reset_index(inplace = True)
#         inherent_risk_main = inherent_risk_main['RULES_TRIGGERED'].to_list()
#         inherent_risk_main = str(inherent_risk_main)
#         inherent_risk_main = inherent_risk_main.replace('\'','')
#         inherent_risk_main = inherent_risk_main.replace('[','')
#         inherent_risk_main = inherent_risk_main.replace(']','')
#         inherent_risk_main = inherent_risk_main.replace(' ','')
#         inherent_risk_main = inherent_risk_main.split(',')
#         inherent_risk_main = [*set(inherent_risk_main)]


#         inherent_risk_main = rules_df[rules_df['RULE_NO'].isin(inherent_risk_main)]['RULE_DESC'].to_list()

#         inherent_risk_other = inherent_df[inherent_df['PERSON_MASTER_ID'].isin(other_entity)]
#         inherent_risk_other.reset_index(inplace = True)
#         inherent_risk_other = inherent_risk_other['RULES_TRIGGERED'].to_list()
#         inherent_risk_other = str(inherent_risk_other)
#         inherent_risk_other = inherent_risk_other.replace('\'','')
#         inherent_risk_other = inherent_risk_other.replace('[','')
#         inherent_risk_other = inherent_risk_other.replace(']','')
#         inherent_risk_other = inherent_risk_other.replace(' ','')
#         inherent_risk_other = inherent_risk_other.split(',')
#         inherent_risk_other = [*set(inherent_risk_other)]


#         inherent_risk_other = rules_df[rules_df['RULE_NO'].isin(inherent_risk_other)]['RULE_DESC'].to_list()


#         all_other_rule = pd.read_sql_query("select * from FINCORE_DB.Fincore_Analytics.RISK_LKP_RULES where person_type = 'Individual' and is_active_flag = 1 and RISK_TYPE  not in ('Auto High Rules ','KYC profile/ Inherent risk based High Risk Indicators - Inherent Risk')",conn)


#         all_other_rule = all_other_rule['RULE_NO'].to_list()
#         all_others_df = []
#         for ind,row in tqdm(risk_person.iterrows()):
#             l = []
#             l = list(row['RISK_RULE_LIST'].split('_'))
#             all_other = row['PERSON_MASTER_ID'],rules_triggered(l,all_other_rule)
#             all_other = list(all_other)
#             all_others_df.append(all_other)


#         all_others_df = pd.DataFrame(all_others_df,columns = ['PERSON_MASTER_ID','RULES_TRIGGERED'])
#         all_others_df['RULES_TRIGGERED']=all_others_df['RULES_TRIGGERED'].astype(str)
#         all_others_df=all_others_df[all_others_df['RULES_TRIGGERED']!='[]']
#         all_others_df.reset_index(inplace = True)
#         all_others_df.drop(columns = ['index'],inplace = True)
#         for ind,row in tqdm(all_others_df.iterrows()):
#             all_others_df['RULES_TRIGGERED'][ind] = all_others_df['RULES_TRIGGERED'][ind].replace('[','')
#             all_others_df['RULES_TRIGGERED'][ind] = all_others_df['RULES_TRIGGERED'][ind].replace(']','')
#             all_others_df['RULES_TRIGGERED'][ind]= all_others_df['RULES_TRIGGERED'][ind].replace('\'','')


#         all_others_df_main = all_others_df[all_others_df['PERSON_MASTER_ID'].isin(main_entity)]
#         all_others_df_main.reset_index(inplace = True)
#         all_others_df_main = all_others_df_main['RULES_TRIGGERED'].to_list()
#         all_others_df_main = str(all_others_df_main)
#         all_others_df_main = all_others_df_main.replace('\'','')
#         all_others_df_main = all_others_df_main.replace('[','')
#         all_others_df_main = all_others_df_main.replace(']','')
#         all_others_df_main = all_others_df_main.replace(' ','')
#         all_others_df_main = all_others_df_main.split(',')
#         all_others_df_main = [*set(all_others_df_main)]

#         all_others_df_main = rules_df[rules_df['RULE_NO'].isin(all_others_df_main)]['RULE_DESC'].to_list()


#         all_others_df_other = all_others_df[all_others_df['PERSON_MASTER_ID'].isin(other_entity)]
#         all_others_df_other.reset_index(inplace = True)
#         all_others_df_other = all_others_df_other['RULES_TRIGGERED'].to_list()
#         all_others_df_other = str(all_others_df_other)
#         all_others_df_other = all_others_df_other.replace('\'','')
#         all_others_df_other = all_others_df_other.replace('[','')
#         all_others_df_other = all_others_df_other.replace(']','')
#         all_others_df_other = all_others_df_other.replace(' ','')
#         all_others_df_other = all_others_df_other.split(',')
#         all_others_df_other = [*set(all_others_df_other)]


#         all_others_df_other = rules_df[rules_df['RULE_NO'].isin(all_others_df_other)]['RULE_DESC'].to_list()




# #         case_entities = pd.read_sql_query("select Case_Master_Id,Entity_Master_Id,Is_Main_Entity_Flag from FINCORE_DB.Fincore.Case_Entity where Is_Active_Flag= 1 and Case_Master_Id = '{}'".format(cmid),conn)



#         s = '<html>'

#         print(all_others_df_main)

#         if (len(main_entity) > 0):
#             if (auto_df_main == []):

#                 if (len(main_entity) == 1):
#                     if (inherent_risk_main == []):
#                         pass
#                     else:
#                         s = s + "<br><ol> The Main entity has <b>" + str(inherent_risk_main) +" </b>."
#                     if (all_others_df_main == []):
#                         pass
#                     else:
#                         s = s + "<br>The Main entity has <b>" + str(all_others_df_main) +"</b>."

#                 else:
#                     if(inherent_risk_main == []):
#                         pass
#                     else:
#                         s = s + "<br><ol> The Main entities have" + str(inherent_risk_main) +"."

#                     if (all_others_df_main == []):
#                         pass  
#                     else:
#                         s = s + "<br> The Main entities have <b>" + str(all_others_df_main) +"</b>."
#                     print(s)



#             else:
#                 if (len(main_entity) == 1):
#                     s = s+"<br></ol> The main entity has <b>"+ str(auto_df_main) + " </b>and is reported on the <b>" + str(main_entity_watchlist)+"</b>watchlist."

#                 elif(len(main_entity) > 1):
#                     s = s+"<br></ol> The main entities have <b>"+ str(auto_df_main) + "</b>and are reported on the <b>" + str(main_entity_watchlist)+"</b>watchlist."


#         else:
#             s = s+"<br></ol> There is no main entity"



#         if (len(other_entity) > 0):

#             if (auto_df_other == []):

#                 if (len(other_entity) == 1):
#                     if (inherent_risk_other == []):
#                         pass
#                     else:
#                         s = s + "<ol> The Other case entity has <b>" + str(inherent_risk_other) +"</b>."

#                     if (all_others_df_other == []):
#                         pass
#                     else:
#                         s = s + "<br> The Other case entity has <b>" + str(all_others_df_other) +"</b>."

#                 else:
#                     if(inherent_risk_other == []):
#                         pass
#                     else:
#                         s = s + "<ol> The Other case entities have <b>" + str(inherent_risk_other) +"</b>."
#                     if (all_others_df_other == []):
#                         pass
#                     else:
#                         s = s + "<br> The Other case entities have <b> " + str(all_others_df_other) +"</b>."


#             else:
#                 if (len(other_entity) == 1):
#                     s = s+"<br></ol> The Other case  entity has <b>"+ str(auto_df_other) + " </b>and is reported on the <b>" + str(other_entity_watchlist)+"</b>watchlist."


#                 elif(len(other_entity) > 1):
#                     s = s+"<br></ol> The Other case  entities have <b>"+ str(auto_df_other) + "</b>and are reported on the <b>" + str(other_entity_watchlist)+"</b>watchlist."



#         else:
#              s = s+"<br></ol> There is no other case entity"
                
                
                
                
                
                
        
        
        
        
        
        



#         CM = pd.read_sql_query("select * from fincore_db.fincore.case_master where case_master_id = '{}' ".format(cmid),conn)
#         RTSS = pd.read_sql_query("select * from fincore_db.fincore_analytics.risk_transaction_summary_stg_TRAN",conn)
#         FP = pd.read_sql_query("select pinCodeNumber,risk_score from DATAHUB_DB.FIUMetaHub.finnet_pincode",conn)
#         CM_RTSS = CM.merge(RTSS,left_on = 'Report_Id',right_on = 'REPORT_ID' ,how = 'inner' )
#         CM_RTSS_FP = CM_RTSS.merge(FP,left_on = 'LOCATION_PINCODE',right_on = 'pinCodeNumber',how = 'inner')
#         CM_RTSS_FP = CM_RTSS_FP[['Case_Master_Id','LOCATION_PINCODE','risk_score']]
#         CM_RTSS_FP.drop_duplicates(inplace = True)
#         CM_RTSS_FP['RANK'] = CM_RTSS_FP.sort_values(['risk_score'],ascending = False).groupby(['Case_Master_Id']).cumcount()+1
#         CM_RTSS_FP['LOCATION_PINCODE'] = CM_RTSS_FP['LOCATION_PINCODE'].astype('int64') 
#         CM_RTSS_FP =  CM_RTSS_FP.query('RANK <= 3')
#         pincode_top_3 = CM_RTSS_FP['LOCATION_PINCODE'].to_list()
#         pincode_top_3 = str(pincode_top_3)
#         pincode_top_3 = pincode_top_3.replace('[','')
#         pincode_top_3 = pincode_top_3.replace(']','',)
        
#         if (len(pincode_top_3) == 0):
#             pass
#         else:
#             s  = s + "<br>High-risk locations of the transactions involved are <b>" + str(pincode_top_3) + ".</b>"




        
        

        
        
        
        


#         case_entities = pd.read_sql_query("select Case_Master_Id,Entity_Master_Id,Is_Main_Entity_Flag from FINCORE_DB.Fincore.Case_Entity where Is_Active_Flag= 1 and Case_Master_Id = '{}'".format(cmid),conn)
#         df = pd.read_sql_query("select * from Fincore_Analytics.RISK_PERSON_SUMMARY_STG_RULES",conn)


#         flagged_person = df[df['RULE_NO'] == 'P001001000']['PERSON_MASTER_ID'].to_list()
#         flagged_person = [*set(flagged_person)]
#         no_of_sanctioned_id = len(intersection(case_entities,flagged_person))
#         if (no_of_sanctioned_id == 0):
#             pass
#         else:
#             s = s+ "<br></ol>This case has <b>"+ str(no_of_sanctioned_id) + "</b> entities on the FIU or LEA watchlist."

#         flagged_person = df[df['RULE_NO'] == 'P001002000']['PERSON_MASTER_ID'].to_list()
#         flagged_person = [*set(flagged_person)]
#         lea_adhoc = len(intersection(case_entities,flagged_person))
        
#         if (lea_adhoc == 0):
#             pass
#         else:
#             s = s+ "<br></ol>LEA Adhoc request has been previously raised on <b> "+ str(lea_adhoc) + "</b> entities."

#         flagged_person = df[df['RULE_NO'] == 'P001003000']['PERSON_MASTER_ID'].to_list()
#         flagged_person = [*set(flagged_person)]
#         international_sanction_list = len(intersection(case_entities,flagged_person))
        
#         if (international_sanction_list == 0):
#             pass
#         else:
#             s = s +'<b></ol>'+ str(international_sanction_list) +'</b>entities are flagged in the international sanctions lists.'
        
#         flagged_person = df[df['RULE_NO'] == 'P001004000']['PERSON_MASTER_ID'].to_list()
#         flagged_person = [*set(flagged_person)]
#         adverse_media_screening = len(intersection(case_entities,flagged_person))
        
#         if (adverse_media_screening == 0):
#             pass
#         else:
#             s = s +'</ol>Adverse Media Screening reveals auto high-risk tags on <b>'+ str(adverse_media_screening) +'</b>entities.'
        
        
        
        
#         flagged_person = df[df['RULE_NO'] == 'P001006000']['PERSON_MASTER_ID'].to_list()
#         flagged_person = [*set(flagged_person)]
#         multiple_pan_din = len(intersection(case_entities,flagged_person))
        
#         if (multiple_pan_din == 0):
#             pass
#         else:
#             s = s +'</ol>Multiple, invalid or wrong PAN/DIN/ DPIN/TAN/IEC number(s) has been reported on <b>'+ str(multiple_pan_din) +'</b>entities.'
        
        
        
        
        
        
        
        
#         s= s+'</html>'

#         s = s.replace('\'','')
        
#         print(s)


#     cursor.execute('''Insert into FINCORE_DB.Fincore_Analytics.risk_summary 
#         VALUES ('{}','{}',GETDATE(),GETDATE())''' .format(cmid,s))
#     conn.commit()
        

# #     print(cmid,s)
        



In [16]:
# MAIN CODE 
# for cmid in cmids:
# #     print(cmid)
#     s = ''

#     case_entities = pd.read_sql_query("select Case_Master_Id,Entity_Master_Id,Is_Main_Entity_Flag from FINCORE_DB.Fincore.Case_Entity where Is_Active_Flag= 1 and Case_Master_Id = '{}'".format(cmid),conn)

#     all_entities = case_entities['Entity_Master_Id'].to_list()


#     main_entity = case_entities[case_entities['Is_Main_Entity_Flag'] == 1]['Entity_Master_Id'].to_list()
#     other_entity = case_entities[case_entities['Is_Main_Entity_Flag'] == 0]['Entity_Master_Id'].to_list()
    
#     if (len(other_entity) > 5):
#         risk_person_top_5 = pd.read_sql_query("SELECT top 5 * FROM FINCORE_ANALYTICS.RISK_PERSON where person_master_id in  {} order by risk_score ".format(str(tuple(other_entity))),conn)
#         other_entity = risk_person_top_5['PERSON_MASTER_ID'].to_list()
        



#     risk_person = pd.read_sql_query("SELECT * FROM FINCORE_ANALYTICS.RISK_PERSON",conn)
    
    
#     RISK_SCORE = risk_person[risk_person['PERSON_MASTER_ID'].isin(main_entity) ]['RISK_SCORE'].to_list()
    
#     try:
#         risk_score_main_entity = RISK_SCORE[0]
#     except IndexError:
#         risk_score_main_entity = 0
        
    
        
#     if risk_score_main_entity < 5:
#         s = s+'<html> The Entity associated with this case has low Risk </html>'
        

    
#     else:
    
#         auto_high_rules = pd.read_sql_query("Select * from fincore_analytics.risk_lkp_rules where RISK_TYPE = 'Auto High Rules ' and IS_ACTIVE_FLAG = 1",conn)
#         auto_high_rules = auto_high_rules['RULE_NO'].to_list()


#         auto_df = []
#         for ind,row in risk_person.iterrows():
#                 l = []
#                 l = list(row['RISK_RULE_LIST'].split('_'))
#                 auto = row['PERSON_MASTER_ID'],rules_triggered(l,auto_high_rules)
#                 auto = list(auto)
#                 auto_df.append(auto)

#         auto_df = pd.DataFrame(auto_df,columns = ['PERSON_MASTER_ID','RULES_TRIGGERED'])
#         auto_df['RULES_TRIGGERED']=auto_df['RULES_TRIGGERED'].astype(str)
#         auto_df=auto_df[auto_df['RULES_TRIGGERED']!='[]']

#         auto_df.reset_index(inplace = True)
#         auto_df.drop(columns = ['index'],inplace = True)


#         for ind,row in auto_df.iterrows():
#             auto_df['RULES_TRIGGERED'][ind] = auto_df['RULES_TRIGGERED'][ind].replace('[','')
#             auto_df['RULES_TRIGGERED'][ind] = auto_df['RULES_TRIGGERED'][ind].replace(']','')
#             auto_df['RULES_TRIGGERED'][ind] = auto_df['RULES_TRIGGERED'][ind].replace('\'','')



#         auto_df_main = auto_df[auto_df['PERSON_MASTER_ID'].isin(main_entity)]
#         auto_df_main.reset_index(inplace = True)
#         auto_df_main = auto_df_main['RULES_TRIGGERED'].to_list()
#         auto_df_main = str(auto_df_main)
#         auto_df_main = auto_df_main.replace('\'','')
#         auto_df_main = auto_df_main.replace('[','')
#         auto_df_main = auto_df_main.replace(']','')
#         auto_df_main = auto_df_main.replace(' ','')
#         auto_df_main = auto_df_main.split(',')
#         auto_df_main = [*set(auto_df_main)]


#         auto_df_main = rules_df[rules_df['RULE_NO'].isin(auto_df_main)]['RULE_DESC'].to_list()


#         auto_df_other = auto_df[auto_df['PERSON_MASTER_ID'].isin(other_entity)]
#         auto_df_other.reset_index(inplace = True)
#         auto_df_other = auto_df_other['RULES_TRIGGERED'].to_list()
#         auto_df_other = str(auto_df_other)
#         auto_df_other = auto_df_other.replace('\'','')
#         auto_df_other = auto_df_other.replace('[','')
#         auto_df_other = auto_df_other.replace(']','')
#         auto_df_other = auto_df_other.replace(' ','')
#         auto_df_other = auto_df_other.split(',')
#         auto_df_other = [*set(auto_df_other)]

#         auto_df_other = rules_df[rules_df['RULE_NO'].isin(auto_df_other)]['RULE_DESC'].to_list()

#         watchlist = pd.read_sql_query("select PERSON_MASTER_ID,RULE_NO,string_agg(RULE_VALUE,',') within group (order by rule_no)  as rule_value from FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG_RULES where RULE_NO in ('P001001000', 'P001003000')group by PERSON_MASTER_ID,RULE_NO",conn)

#         main_entity_watchlist = watchlist[watchlist['PERSON_MASTER_ID'].isin(main_entity)]
#         main_entity_watchlist.reset_index(inplace = True)

#         main_entity_watchlist = main_entity_watchlist['RULE_NO'].to_list()
#         main_entity_watchlist = str(main_entity_watchlist)
#         main_entity_watchlist = main_entity_watchlist.replace('\'','')
#         main_entity_watchlist = main_entity_watchlist.replace('[','')
#         main_entity_watchlist = main_entity_watchlist.replace(']','')
#         main_entity_watchlist = main_entity_watchlist.replace(' ','')
#         main_entity_watchlist = main_entity_watchlist.split(',')
#         main_entity_watchlist = [*set(main_entity_watchlist)]

#         other_entity_watchlist = watchlist[watchlist['PERSON_MASTER_ID'].isin(other_entity)]
#         other_entity_watchlist.reset_index(inplace = True)
#         other_entity_watchlist = other_entity_watchlist['RULE_NO'].to_list()
#         other_entity_watchlist = str(other_entity_watchlist)
#         other_entity_watchlist = other_entity_watchlist.replace('\'','')
#         other_entity_watchlist = other_entity_watchlist.replace('[','')
#         other_entity_watchlist = other_entity_watchlist.replace(']','')
#         other_entity_watchlist = other_entity_watchlist.replace(' ','')
#         other_entity_watchlist = other_entity_watchlist.split(',')
#         other_entity_watchlist = [*set(other_entity_watchlist)]


#         inherent_risk = pd.read_sql_query("select * from FINCORE_DB.Fincore_Analytics.RISK_LKP_RULES where person_type = 'Individual' and is_active_flag = 1 and RISK_TYPE = 'KYC profile/ Inherent risk based High Risk Indicators - Inherent Risk'",conn)


#         inherent_risk = inherent_risk['RULE_NO'].to_list()



#         inherent_df = []
#         for ind,row in risk_person.iterrows():
#                 l = []
#                 l = list(row['RISK_RULE_LIST'].split('_'))
#                 inherent = row['PERSON_MASTER_ID'],rules_triggered(l,inherent_risk)
#                 inherent = list(inherent)
#                 inherent_df.append(inherent)



#         inherent_df = pd.DataFrame(inherent_df,columns = ['PERSON_MASTER_ID','RULES_TRIGGERED'])
#         inherent_df['RULES_TRIGGERED']=inherent_df['RULES_TRIGGERED'].astype(str)
#         inherent_df=inherent_df[inherent_df['RULES_TRIGGERED']!='[]']
#         inherent_df.reset_index(inplace = True)
#         inherent_df.drop(columns = ['index'],inplace = True)


#         for ind,row in inherent_df.iterrows():
#             inherent_df['RULES_TRIGGERED'][ind] = inherent_df['RULES_TRIGGERED'][ind].replace('[','')
#             inherent_df['RULES_TRIGGERED'][ind] = inherent_df['RULES_TRIGGERED'][ind].replace(']','')
#             inherent_df['RULES_TRIGGERED'][ind]=inherent_df['RULES_TRIGGERED'][ind].replace('\'','')



#         inherent_risk_main = inherent_df[inherent_df['PERSON_MASTER_ID'].isin(main_entity)]
#         inherent_risk_main.reset_index(inplace = True)
#         inherent_risk_main = inherent_risk_main['RULES_TRIGGERED'].to_list()
#         inherent_risk_main = str(inherent_risk_main)
#         inherent_risk_main = inherent_risk_main.replace('\'','')
#         inherent_risk_main = inherent_risk_main.replace('[','')
#         inherent_risk_main = inherent_risk_main.replace(']','')
#         inherent_risk_main = inherent_risk_main.replace(' ','')
#         inherent_risk_main = inherent_risk_main.split(',')
#         inherent_risk_main = [*set(inherent_risk_main)]


#         inherent_risk_main = rules_df[rules_df['RULE_NO'].isin(inherent_risk_main)]['RULE_DESC'].to_list()

#         inherent_risk_other = inherent_df[inherent_df['PERSON_MASTER_ID'].isin(other_entity)]
#         inherent_risk_other.reset_index(inplace = True)
#         inherent_risk_other = inherent_risk_other['RULES_TRIGGERED'].to_list()
#         inherent_risk_other = str(inherent_risk_other)
#         inherent_risk_other = inherent_risk_other.replace('\'','')
#         inherent_risk_other = inherent_risk_other.replace('[','')
#         inherent_risk_other = inherent_risk_other.replace(']','')
#         inherent_risk_other = inherent_risk_other.replace(' ','')
#         inherent_risk_other = inherent_risk_other.split(',')
#         inherent_risk_other = [*set(inherent_risk_other)]


#         inherent_risk_other = rules_df[rules_df['RULE_NO'].isin(inherent_risk_other)]['RULE_DESC'].to_list()


#         all_other_rule = pd.read_sql_query("select * from FINCORE_DB.Fincore_Analytics.RISK_LKP_RULES where person_type = 'Individual' and is_active_flag = 1 and RISK_TYPE  not in ('Auto High Rules ','KYC profile/ Inherent risk based High Risk Indicators - Inherent Risk')",conn)


#         all_other_rule = all_other_rule['RULE_NO'].to_list()
#         all_others_df = []
#         for ind,row in tqdm(risk_person.iterrows()):
#             l = []
#             l = list(row['RISK_RULE_LIST'].split('_'))
#             all_other = row['PERSON_MASTER_ID'],rules_triggered(l,all_other_rule)
#             all_other = list(all_other)
#             all_others_df.append(all_other)


#         all_others_df = pd.DataFrame(all_others_df,columns = ['PERSON_MASTER_ID','RULES_TRIGGERED'])
#         all_others_df['RULES_TRIGGERED']=all_others_df['RULES_TRIGGERED'].astype(str)
#         all_others_df=all_others_df[all_others_df['RULES_TRIGGERED']!='[]']
#         all_others_df.reset_index(inplace = True)
#         all_others_df.drop(columns = ['index'],inplace = True)
#         for ind,row in tqdm(all_others_df.iterrows()):
#             all_others_df['RULES_TRIGGERED'][ind] = all_others_df['RULES_TRIGGERED'][ind].replace('[','')
#             all_others_df['RULES_TRIGGERED'][ind] = all_others_df['RULES_TRIGGERED'][ind].replace(']','')
#             all_others_df['RULES_TRIGGERED'][ind]= all_others_df['RULES_TRIGGERED'][ind].replace('\'','')


#         all_others_df_main = all_others_df[all_others_df['PERSON_MASTER_ID'].isin(main_entity)]
#         all_others_df_main.reset_index(inplace = True)
#         all_others_df_main = all_others_df_main['RULES_TRIGGERED'].to_list()
#         all_others_df_main = str(all_others_df_main)
#         all_others_df_main = all_others_df_main.replace('\'','')
#         all_others_df_main = all_others_df_main.replace('[','')
#         all_others_df_main = all_others_df_main.replace(']','')
#         all_others_df_main = all_others_df_main.replace(' ','')
#         all_others_df_main = all_others_df_main.split(',')
#         all_others_df_main = [*set(all_others_df_main)]

#         all_others_df_main = rules_df[rules_df['RULE_NO'].isin(all_others_df_main)]['RULE_DESC'].to_list()


#         all_others_df_other = all_others_df[all_others_df['PERSON_MASTER_ID'].isin(other_entity)]
#         all_others_df_other.reset_index(inplace = True)
#         all_others_df_other = all_others_df_other['RULES_TRIGGERED'].to_list()
#         all_others_df_other = str(all_others_df_other)
#         all_others_df_other = all_others_df_other.replace('\'','')
#         all_others_df_other = all_others_df_other.replace('[','')
#         all_others_df_other = all_others_df_other.replace(']','')
#         all_others_df_other = all_others_df_other.replace(' ','')
#         all_others_df_other = all_others_df_other.split(',')
#         all_others_df_other = [*set(all_others_df_other)]


#         all_others_df_other = rules_df[rules_df['RULE_NO'].isin(all_others_df_other)]['RULE_DESC'].to_list()




# #         case_entities = pd.read_sql_query("select Case_Master_Id,Entity_Master_Id,Is_Main_Entity_Flag from FINCORE_DB.Fincore.Case_Entity where Is_Active_Flag= 1 and Case_Master_Id = '{}'".format(cmid),conn)



#         s = '<html>'

#         print(all_others_df_main)

#         if (len(main_entity) > 0):
#             if (auto_df_main == []):

#                 if (len(main_entity) == 1):
#                     if (inherent_risk_main == []):
#                         pass
#                     else:
#                         s = s + "<br><ol> The Main entity has <b>" + str(inherent_risk_main) +" </b>."
#                     if (all_others_df_main == []):
#                         pass
#                     else:
#                         s = s + "<br>The Main entity has <b>" + str(all_others_df_main) +"</b>."

#                 else:
#                     if(inherent_risk_main == []):
#                         pass
#                     else:
#                         s = s + "<br><ol> The Main entities have" + str(inherent_risk_main) +"."

#                     if (all_others_df_main == []):
#                         pass  
#                     else:
#                         s = s + "<br> The Main entities have <b>" + str(all_others_df_main) +"</b>."
#                     print(s)



#             else:
#                 if (len(main_entity) == 1):
#                     s = s+"<br></ol> The main entity has <b>"+ str(auto_df_main) + " </b>and is reported on the <b>" + str(main_entity_watchlist)+"</b>watchlist."

#                 elif(len(main_entity) > 1):
#                     s = s+"<br></ol> The main entities have <b>"+ str(auto_df_main) + "</b>and are reported on the <b>" + str(main_entity_watchlist)+"</b>watchlist."


#         else:
#             s = s+"<br></ol> There is no main entity"



#         if (len(other_entity) > 0):

#             if (auto_df_other == []):

#                 if (len(other_entity) == 1):
#                     if (inherent_risk_other == []):
#                         pass
#                     else:
#                         s = s + "<ol> The Other case entity has <b>" + str(inherent_risk_other) +"</b>."

#                     if (all_others_df_other == []):
#                         pass
#                     else:
#                         s = s + "<br> The Other case entity has <b>" + str(all_others_df_other) +"</b>."

#                 else:
#                     if(inherent_risk_other == []):
#                         pass
#                     else:
#                         s = s + "<ol> The Other case entities have <b>" + str(inherent_risk_other) +"</b>."
#                     if (all_others_df_other == []):
#                         pass
#                     else:
#                         s = s + "<br> The Other case entities have <b> " + str(all_others_df_other) +"</b>."


#             else:
#                 if (len(other_entity) == 1):
#                     s = s+"<br></ol> The Other case  entity has <b>"+ str(auto_df_other) + " </b>and is reported on the <b>" + str(other_entity_watchlist)+"</b>watchlist."


#                 elif(len(other_entity) > 1):
#                     s = s+"<br></ol> The Other case  entities have <b>"+ str(auto_df_other) + "</b>and are reported on the <b>" + str(other_entity_watchlist)+"</b>watchlist."



#         else:
#              s = s+"<br></ol> There is no other case entity"
                
                
                
                
                
                
        
        
        
        
        
        



#         CM = pd.read_sql_query("select * from fincore_db.fincore.case_master where case_master_id = '{}' ".format(cmid),conn)
#         RTSS = pd.read_sql_query("select * from fincore_db.fincore_analytics.risk_transaction_summary_stg_TRAN",conn)
#         FP = pd.read_sql_query("select pinCodeNumber,risk_score from DATAHUB_DB.FIUMetaHub.finnet_pincode",conn)
#         CM_RTSS = CM.merge(RTSS,left_on = 'Report_Id',right_on = 'REPORT_ID' ,how = 'inner' )
#         CM_RTSS_FP = CM_RTSS.merge(FP,left_on = 'LOCATION_PINCODE',right_on = 'pinCodeNumber',how = 'inner')
#         CM_RTSS_FP = CM_RTSS_FP[['Case_Master_Id','LOCATION_PINCODE','risk_score']]
#         CM_RTSS_FP.drop_duplicates(inplace = True)
#         CM_RTSS_FP['RANK'] = CM_RTSS_FP.sort_values(['risk_score'],ascending = False).groupby(['Case_Master_Id']).cumcount()+1
#         CM_RTSS_FP['LOCATION_PINCODE'] = CM_RTSS_FP['LOCATION_PINCODE'].astype('int64') 
#         CM_RTSS_FP =  CM_RTSS_FP.query('RANK <= 3')
#         pincode_top_3 = CM_RTSS_FP['LOCATION_PINCODE'].to_list()
#         pincode_top_3 = str(pincode_top_3)
#         pincode_top_3 = pincode_top_3.replace('[','')
#         pincode_top_3 = pincode_top_3.replace(']','',)
        
#         if (len(pincode_top_3) == 0):
#             pass
#         else:
#             s  = s + "<br>High-risk locations of the transactions involved are <b>" + str(pincode_top_3) + ".</b>"




        
        

        
        
        
        


#         case_entities = pd.read_sql_query("select Case_Master_Id,Entity_Master_Id,Is_Main_Entity_Flag from FINCORE_DB.Fincore.Case_Entity where Is_Active_Flag= 1 and Case_Master_Id = '{}'".format(cmid),conn)
#         df = pd.read_sql_query("select * from Fincore_Analytics.RISK_PERSON_SUMMARY_STG_RULES",conn)


#         flagged_person = df[df['RULE_NO'] == 'P001001000']['PERSON_MASTER_ID'].to_list()
#         flagged_person = [*set(flagged_person)]
#         no_of_sanctioned_id = len(intersection(case_entities,flagged_person))
#         if (no_of_sanctioned_id == 0):
#             pass
#         else:
#             s = s+ "<br></ol>This case has <b>"+ str(no_of_sanctioned_id) + "</b> entities on the FIU or LEA watchlist."

#         flagged_person = df[df['RULE_NO'] == 'P001002000']['PERSON_MASTER_ID'].to_list()
#         flagged_person = [*set(flagged_person)]
#         lea_adhoc = len(intersection(case_entities,flagged_person))
        
#         if (lea_adhoc == 0):
#             pass
#         else:
#             s = s+ "<br></ol>LEA Adhoc request has been previously raised on <b> "+ str(lea_adhoc) + "</b> entities."

#         flagged_person = df[df['RULE_NO'] == 'P001003000']['PERSON_MASTER_ID'].to_list()
#         flagged_person = [*set(flagged_person)]
#         international_sanction_list = len(intersection(case_entities,flagged_person))
        
#         if (international_sanction_list == 0):
#             pass
#         else:
#             s = s +'<b></ol>'+ str(international_sanction_list) +'</b>entities are flagged in the international sanctions lists.'
        
#         flagged_person = df[df['RULE_NO'] == 'P001004000']['PERSON_MASTER_ID'].to_list()
#         flagged_person = [*set(flagged_person)]
#         adverse_media_screening = len(intersection(case_entities,flagged_person))
        
#         if (adverse_media_screening == 0):
#             pass
#         else:
#             s = s +'</ol>Adverse Media Screening reveals auto high-risk tags on <b>'+ str(adverse_media_screening) +'</b>entities.'
        
        
        
        
#         flagged_person = df[df['RULE_NO'] == 'P001006000']['PERSON_MASTER_ID'].to_list()
#         flagged_person = [*set(flagged_person)]
#         multiple_pan_din = len(intersection(case_entities,flagged_person))
        
#         if (multiple_pan_din == 0):
#             pass
#         else:
#             s = s +'</ol>Multiple, invalid or wrong PAN/DIN/ DPIN/TAN/IEC number(s) has been reported on <b>'+ str(multiple_pan_din) +'</b>entities.'
        
        
        
        
        
        
        
        
#         s= s+'</html>'

#         s = s.replace('\'','')
        
#         print(s)


#     cursor.execute('''Insert into FINCORE_DB.Fincore_Analytics.risk_summary 
#         VALUES ('{}','{}',GETDATE(),GETDATE())''' .format(cmid,s))
#     conn.commit()
        

# #     print(cmid,s)
        



In [17]:
cursor.execute(f'''Update FINCORE_DB.Fincore_Analytics.AUDIT_INTGN set JOB_END_DATE = Getdate(),JOB_STATUS = 'Success',NO_OF_RECORD = {no_of_records} where  JOB_RUN_ID ={job_id}''')

cursor.commit()


In [18]:
conn.close()
